# `Практикум по программированию на языке Python`

## `Задание 03. Рекуррентные Нейронные Сети. Dropout. LM`

#### Фамилия, имя: Богачев Владимир

Дата выдачи: <span style="color:red">__30 марта 23:59__</span>.

Мягкий дедлайн: <span style="color:red">__13 апреля 23:59__</span>.

Стоимость: __10 баллов__ (основная часть заданий) + __7 баллов__ (дополнительные задания).

<span style="color:red">__В ноутбуке все клетки должны выполняться без ошибок при последовательном их выполнении.__</span>

#### `Москва, 2024`

Данное задание будет состоять из двух частей:
1. Применение рекуррентной сети для решения задачи классификации текста. Более конкретно -- предсказания рейтинга отзыва фильма.
2. Простейшая лингвистическая модель для генерации текста на основе LSTM.

При выполнении задания вы обучите LSTM с разным уровнем "коробочности", а также познакомитесь с различными способами применения DropOut к рекуррентным архитектурам. В рекуррентных архитектурах вариантов, куда можно наложить бинарную маску шума, гораздо больше, чем в нейросетях прямого прохода.

Во второй части вы попробуете реализовать простейший рекуррентный декодер для генерации текстов.

Задание сделано так, чтобы его можно было выполнять на CPU, однако RNN - это ресурсоёмкая вещь, поэтому на GPU с ними работать приятнее. Можете попробовать использовать [https://colab.research.google.com](https://colab.research.google.com) - бесплатное облако с GPU.

**Для корректного отображения картинок, вам может понадобится сделать ноутбук доверенным (Trusted) в правом верхнем углу**

# `Часть 0. Загрузка и предобработка данных (1 балл)`

## `Рекомендуемые гиперпараметры`

In [1]:
max_length = 200
top_n_words = 5000

hidden_dim = 128
embedding_dim = 32

num_epochs = 15
batch_size = 64
learning_rate = 1e-3

Первое, что нужно сделать — скачать, предобработать данные и организовать их таким образом, чтобы их можно было подавать в нейронную сеть.

Для обеих частей задания мы будем использовать [**Large Movie Review Dataset**](https://ai.stanford.edu/~amaas/data/sentiment/).

## `Загрузка и предобработка данных`

Загрузите данные по ссылке выше. (**tip**: используйте `wget`)

In [2]:
!wget https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz

"wget" ­Ґ пў«пҐвбп ў­гваҐ­­Ґ© Ё«Ё ў­Ґи­Ґ©
Є®¬ ­¤®©, ЁбЇ®«­пҐ¬®© Їа®Ја ¬¬®© Ё«Ё Ї ЄҐв­л¬ д ©«®¬.


Распакуйте скачанные данные в папку `aclImdb` (**tip:** используйте `tar`)

In [3]:
!tar -xzf aclImdb_v1.tar.gz

Посмотрите в файле `./aclImdb/README` как организованы данные:

In [4]:
!cat ./aclImdb/train/pos/10003_8.txt

"cat" ­Ґ пў«пҐвбп ў­гваҐ­­Ґ© Ё«Ё ў­Ґи­Ґ©
Є®¬ ­¤®©, ЁбЇ®«­пҐ¬®© Їа®Ја ¬¬®© Ё«Ё Ї ЄҐв­л¬ д ©«®¬.


In [5]:
test_data_path = './aclImdb/test/'
train_data_path = './aclImdb/train/'

In [6]:
import os
from functools import partial
from collections import defaultdict

from IPython.display import Markdown, display

import nltk
nltk.download('stopwords')

import regex
import numpy as np

import torch
import torchtext
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F

torch.backends.cudnn.benchmark = True
torch.use_deterministic_algorithms(False)

torch.autograd.profiler.profile(False)
torch.autograd.profiler.emit_nvtx(False)
torch.autograd.set_detect_anomaly(False)

torch.set_float32_matmul_precision('high')
torch.backends.cuda.matmul.allow_tf32 = True

[nltk_data] Error loading stopwords: <urlopen error [WinError 10060]
[nltk_data]     Попытка установить соединение была безуспешной, т.к.
[nltk_data]     от другого компьютера за требуемое время не получен
[nltk_data]     нужный отклик, или было разорвано уже установленное
[nltk_data]     соединение из-за неверного отклика уже подключенного
[nltk_data]     компьютера>


Стандартной предобработкой данных является токенизация текстов. Полученные токены можно будет закодировать и затем подавать на вход нейронной сети. Ключевым моментом, который влияет на скорость работы нейросети и её размер в памяти — размер словаря, используемого при токенизации. Для задачи классификации мы можем убрать часть слов (стоп слова, редкие слова), ускорив обучение без потери в качестве.

In [7]:
torch.cuda.is_available()

True

In [8]:
STOPWORDS = nltk.corpus.stopwords.words('english')

Реализуйте функцию для токенизации текста. Выполнять токенизацию можно по-разному, но в данном задании предлагается это делать следующим образом:
1. Привести текст к нижнему регистру
2. Убрать html разметку из текстов (`<br />`, ...)
3. Убрать все символы кроме латинских букв
4. Разбить строку по пробелам
5. Убрать стоп слова

In [9]:
import string
from nltk.tokenize import word_tokenize
from bs4 import BeautifulSoup

In [10]:
translator = str.maketrans('', '', string.punctuation + string.digits)

def tokenize(text):
    """
    :param str text: Input text 
    :return List[str]: List of words
    """
    text = text.lower()
    text = BeautifulSoup(text).get_text()
    text = text.translate(translator)
    text = word_tokenize(text, language='english')
    text = list(filter(lambda w: w not in STOPWORDS, text))
    
    return text

In [11]:
tokenize('1. Hello <br /> words!! <br />')

['hello', 'words']

Теперь мы можем создать словарь, с помощью которого мы будем численно кодировать токены из текста и наоборот.

Удобной обёрткой для создания словарей является класс `torchtext.vocab.Vocab` и фабрика для создания таких классов `torchtext.vocab.vocab`.

In [12]:
torchtext.vocab.vocab??

Чтобы создать такой словарь, сначала нужно создать словарь со всеми токенами в тексте и их частотами встречаемости:

In [13]:
counter = defaultdict(int)

for path in ['./aclImdb/test/neg', './aclImdb/test/pos', './aclImdb/train/neg', './aclImdb/train/pos']:
    for file_path in os.listdir(path):
        text = open(os.path.join(path, file_path), 'r', encoding='utf-8', errors='ignore').read().strip()
        for token in tokenize(text):
            counter[token] += 1

C:\Users\Vladimir\AppData\Local\Temp\ipykernel_12668\3422549362.py:9: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  text = BeautifulSoup(text).get_text()


Для работы с текстами нам необходимо зарезервировать два специальных токена:
1. `<pad>` для токена означающего паддинг
2. `<unk>` для токенов, которые отсутствуют в словаре

In [14]:
# specials = ['<pad>', '<unk>']
specials = ['<pad>', '<unk>', '<sos>', '<eos>']
for special in specials:
    counter[special] = 0

Создайте словарь из словаря частот `counter`. Наименьшие *id* отдайте под специальные токены. 

Отбросьте низкочастотные слова, оставив только `top_n_words` слов. Можете использовать любой способ реализации этого условия, например:
1. Оставить в словаре `counter` нужное число слов
2. Подобрать параметр `min_freq`, чтобы оставшееся число слов было близко к необходимому порогу

In [15]:
torchtext.__version__

'0.16.0+cpu'

In [16]:
vocab = torchtext.vocab.vocab(
    counter,
    min_freq=145,
    specials=specials,
)

vocab.set_default_index(vocab['<unk>'])

In [17]:
vocab, len(vocab)

(Vocab(), 5037)

In [18]:
vocab.lookup_indices(['<pad>', '<unk>'])

[0, 1]

In [19]:
vocab.lookup_indices(['this', 'film', 'was', 'awful'])

[1, 98, 1, 422]

Теперь мы готовы создать обёртку-датасет для наших данных. 

Необходимо добавить несколько опции, которые понадобятся во второй части задания:
1. Ограничение на максимальную длину текста в токенах. Если текст оказывается длиннее, то последние токены отбрасываются
2. Возможность добавить в специальные токены `<sos>`, `<eos>` в начало и конец токенизированного текста
    
**tips:**
1. Обратите особое внимание, что у длинных текстов не должен обрезаться паддинг
2. В исходных данных рейтинг закодирован в названии файла в виде числа от $1$ до $10$. Для удобства, вычтите $1$, чтобы рейтинг был от $0$ до $9$

In [20]:
import re

In [21]:
re_rating = re.compile('_[0-9]*')
re.search(re_rating, './aclImdb/train/pos/10003_10.txt').group(0)[1:]

'10'

In [22]:
def get_raiting(path: str, re_rating = None):
    re_rating = '_[0-9]*' if re_rating is None else re_rating
    
    rating = re.search(re_rating, path).group(0)[1:]
    return int(rating) - 1

In [23]:
class LargeMovieReviewDataset(Dataset):
    def __init__(self, data_path, vocab, max_len, pad_sos=False, pad_eos=False):
        """
        :param str data_path: Path to folder with one of the data splits (train or test)
        :param torchtext.vocab.Vocab vocab: dictionary with lookup_indices method
        :param int max_len: Maximum length of tokenized text
        :param bool pad_sos: If True pad sequence at the beginning with <sos> 
        :param bool pad_eos: If True pad sequence at the end with <eos>         
        """
        super().__init__()
        
        self.pad_sos = pad_sos
        if self.pad_sos:
            self.sos_id = vocab.lookup_indices(['<sos>'])[0]
        self.pad_eos = pad_eos
        if self.pad_eos:
            self.eos_id = vocab.lookup_indices(['<eos>'])[0]
        
        self.vocab = vocab
        self.max_len = max_len
        self.data_path = data_path
        self.negative_path = os.path.join(data_path, 'neg')
        self.positive_path = os.path.join(data_path, 'pos')
        
        self.negative_paths = []
        self.positive_paths = []

        for file_path in os.listdir(self.negative_path):
            self.negative_paths.append(os.path.join(self.negative_path, file_path))

        for file_path in os.listdir(self.positive_path):
            self.positive_paths.append(os.path.join(self.positive_path, file_path))
        
        self.texts = []
        self.tokens = []
        self.ratings = []
        self.labels = [0] * len(self.negative_paths) + [1] * len(self.positive_paths)
        
        # Read each file in data_path, tokenize it, get tokens ids, its rating and store
        re_rating = re.compile('_[0-9]*')
        for path in self.negative_paths + self.positive_paths:
            # YOUR CODE HERE
            with open(path, "r", encoding='utf-8', errors='ignore') as f:
                text = f.read().strip()
            self.texts.append(text)
            
            txt_tokens = vocab.lookup_indices(tokenize(text))[0:self.max_len]
            if self.pad_sos:
                txt_tokens.insert(0, vocab['<sos>'])
            if self.pad_eos:
                txt_tokens.append(vocab['<eos>'])
            
            self.tokens.append(txt_tokens)
            self.ratings.append(get_raiting(path, re_rating=re_rating))
        
        self.ratings = torch.LongTensor(self.ratings)
        self.labels = torch.LongTensor(self.labels)
        self.tokens = [torch.LongTensor(tls) for tls in self.tokens]
        
        
    def __getitem__(self, idx):
        """
        :param int idx: index of object in dataset
        :return dict: Dictionary with all useful object data 
            {
                'text' str: unprocessed text,
                'label' torch.Tensor(dtype=torch.long): sentiment of the text (0 for negative, 1 for positive)
                'rating' torch.Tensor(dtype=torch.long): rating of the text
                'tokens' torch.Tensor(dtype=torch.long): tensor of tokens ids for the text
                'tokens_len' torch.Tensor(dtype=torch.long): number of tokens
            }
        """
        # YOUR CODE HERE
        
        res = {
            'text': self.texts[idx],
            'label': self.labels[idx],
            'rating': self.ratings[idx],
            'tokens': self.tokens[idx],
            'tokens_len': self.tokens[idx].shape[0]
        }
        
        return res
    
    def __len__(self):
        """
        :return int: number of objects in dataset 
        """
        return len(self.tokens)

Создайте датасеты для тестовой и обучающей выборки. 

Обратите внимание, что для задачи классификации нам не потребуется паддинг с помощью `<sos>`, `<eos>`. 

Не забудьте обрезать длинные тексты, передав параметр `max_length`.

In [24]:
import pathlib

In [25]:
datasets_dump_path = pathlib.Path("./datasets_dump.bin")

if not datasets_dump_path.exists():
    test_dataset = LargeMovieReviewDataset(test_data_path, vocab, max_len=max_length)
    train_dataset = LargeMovieReviewDataset(train_data_path, vocab, max_len=max_length)
    
    with open(datasets_dump_path, "wb") as f:
        torch.save(
            obj=(test_dataset, train_dataset),
            f=f
        )
else:
    with open(datasets_dump_path, "rb") as f:
        test_dataset, train_dataset = torch.load(f)

In [26]:
test_dataset[0]

{'text': "Once again Mr. Costner has dragged out a movie for far longer than necessary. Aside from the terrific sea rescue sequences, of which there are very few I just did not care about any of the characters. Most of us have ghosts in the closet, and Costner's character are realized early on, and then forgotten until much later, by which time I did not care. The character we should really care about is a very cocky, overconfident Ashton Kutcher. The problem is he comes off as kid who thinks he's better than anyone else around him and shows no signs of a cluttered closet. His only obstacle appears to be winning over Costner. Finally when we are well past the half way point of this stinker, Costner tells us all about Kutcher's ghosts. We are told why Kutcher is driven to be the best with no prior inkling or foreshadowing. No magic here, it was all I could do to keep from turning it off an hour in.",
 'label': tensor(0),
 'rating': tensor(1),
 'tokens': tensor([ 4,  1,  5,  6,  7,  8,  

Посмотрим, как выглядит объект в датасете:

In [27]:
for d in train_dataset:
    # print(type(d['tokens']))
    # break
    assert not torch.any(torch.isnan(d['tokens'])), f"tokens contains NaNs: \n{d=}"
    
for d in test_dataset:
    # print(type(d['tokens']))
    # break
    assert not torch.any(torch.isnan(d['tokens'])), f"tokens contains NaNs: \n{d=}"

In [28]:
test_dataset[-2]

{'text': "This movie, with all its complexity and subtlety, makes for one of the most thought-provoking short films I have ever seen. The topics it addresses are ugly, cynical, and at times, even macabre, but the film remains beautiful in its language, artful with its camera angles, and gorgeous in its style, skillfully recreating the short story of the same name written by a master of short stories, Tobias Wolff.<br /><br />Not wishing to spoil anything of the movie, I won't go into any details, other than to say that this movie is magnificent in and of itself. It takes pride in what it does, and does it well. It shows the most important memories of life, all of which can be topped by the single most elusive feeling: unexpected bliss. This movie, of its own volition, has created in me the same feelings the main character (Tom Noonan) felt when words transformed his very existence, and that is one impressive feat.",
 'label': tensor(1),
 'rating': tensor(9),
 'tokens': tensor([   6, 36

Теперь нам нужно создать `DataLoader` для наших данных. `DataLoader` умеет из коробки объединять список объектов из датасета в один батч, даже когда датасет возвращает словарь тензоров. Однако, это работает только в случае когда все эти тензоры имеют один и тот же размер во всех батчах. В нашем случае, это не так, так как разные тексты могут иметь разную длину.

Чтобы обойти эту проблему у `DataLoader` есть параметр `collate_fn`, который позволяет задать функцию для объединения списка объектов в один батч.

Чтобы объединить несколько тензоров разной длины в один можно использовать функцию `torch.nn.utils.rnn.pad_sequence`

Обратите внимание на её аргументы:
1. `batch_first` определяет по какой оси "складывать" тензоры. Предпочтительнее использовать `batch_first=False` так как это может упростить выполнение задания в дальнейшем 
2. `padding_value` — число, которое будет использоваться в качестве паддинга, чтобы сделать все тензоры одинаковой длины

In [29]:
torch.nn.utils.rnn.pad_sequence([
    torch.tensor([1, 2, 3]),
    torch.tensor([4, 5]),
    torch.tensor([6, 7, 8, 9])
], batch_first=False, padding_value=-1)

tensor([[ 1,  4,  6],
        [ 2,  5,  7],
        [ 3, -1,  8],
        [-1, -1,  9]])

In [30]:
def collate_fn(batch, padding_value, batch_first=False):
    """
    :param List[Dict] batch: List of objects from dataset
    :param int padding_value: Value that will be used to pad tokens
    :param bool batch_first: If True resulting tensor with tokens must have shape [B, T] otherwise [T, B]
    :return dict: Dictionary with all data collated
        {
            'ratings' torch.Tensor(dtype=torch.long): rating of the text for each object in batch
            'labels' torch.Tensor(dtype=torch.long): sentiment of the text for each object in batch
            
            'texts' List[str]: All texts in one list
            'tokens' torch.Tensor(dtype=torch.long): tensor of tokens ids padded with @padding_value
            'tokens_lens' torch.Tensor(dtype=torch.long): number of tokens for each object in batch
        }
    """
    ratings = torch.LongTensor(size=(len(batch), ))
    labels = torch.LongTensor(size=(len(batch), ))
    texts = []
    tokens = []
    tokens_lens = torch.LongTensor(size=(len(batch), ))
    
    for i, batch_dir in enumerate(batch):
        ratings[i] = batch_dir['rating']
        labels[i] = batch_dir['label']
        texts.append(batch_dir['text'])
        tokens.append(batch_dir['tokens'])
        tokens_lens[i] = batch_dir['tokens_len']
    
    tokens = torch.nn.utils.rnn.pad_sequence(tokens, batch_first=batch_first, padding_value=padding_value)
    
    res = {
            'texts': texts,
            'labels': labels,
            'ratings': ratings,
            'tokens': tokens,
            'tokens_lens': tokens_lens,
        }
    
    return res

Создайте даталоадеры с использованием `collate_fn`.

**tips**:
1. Передать в `collate_fn` правильное значение паддинга можно, например, с помощью `functools.partial`
2. Если вы работаете в Google Colab, то, возможно, вам будет необходимо установить `num_workers=0` во избежание падения ноутбука.

In [31]:
import functools

In [32]:
collate_fn_ = functools.partial(collate_fn, padding_value=vocab['<pad>'])
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, drop_last=False, num_workers=0, collate_fn=collate_fn_)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True, num_workers=0, collate_fn=collate_fn_)

In [33]:
for d in train_dataloader:
    # print(type(d['tokens']))
    # break
    assert not torch.any(torch.isnan(d['tokens'])), f"tokens contains NaNs: \n{d=}"

for d in test_dataloader:
    # print(type(d['tokens']))
    # break
    assert not torch.any(torch.isnan(d['tokens'])), f"tokens contains NaNs: \n{d=}"

Посмотрим на какой-нибудь батч:

In [34]:
batch = next(iter(test_dataloader))
batch.keys(), batch['labels'], batch['ratings'], batch['tokens'], batch['tokens_lens']

(dict_keys(['texts', 'labels', 'ratings', 'tokens', 'tokens_lens']),
 tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 tensor([1, 3, 0, 2, 2, 1, 1, 1, 3, 3, 2, 2, 1, 0, 0, 3, 1, 3, 2, 0, 0, 0, 3, 0,
         2, 3, 3, 2, 1, 2, 0, 2, 0, 2, 2, 1, 0, 0, 0, 0, 0, 0, 3, 0, 3, 1, 0, 0,
         0, 1, 0, 3, 0, 0, 0, 3, 3, 1, 3, 3, 2, 0, 0, 0]),
 tensor([[   4,   58,  136,  ..., 1984,  514,    1],
         [   1,   59,  137,  ...,  284,    1,  274],
         [   5,   60,  138,  ...,  349,    1, 2002],
         ...,
         [   0,    0,    0,  ...,    0,    0,    0],
         [   0,    0,    0,  ...,    0,    0,    0],
         [   0,    0,    0,  ...,    0,    0,    0]]),
 tensor([ 74, 128, 108, 168, 137,  52,  74,  74,  72,  98,  59, 143, 134,  52,
         104, 112,  67, 116, 189,  47,  36,  96, 200, 200, 136, 111, 105, 20

# `Часть 1. Классификация текстов (4 балла)`

## `Сборка и обучение RNN в pytorch (1 балл)`

Создадим переменные для device-agnostic кода:

In [35]:
dtype, device, cuda_device_id = torch.float32, None, 0
os.environ["CUDA_VISIBLE_DEVICES"] = '{0}'.format(str(cuda_device_id) if cuda_device_id is not None else '')
if cuda_device_id is not None and torch.cuda.is_available():
    device = 'cuda:{0:d}'.format(0)
else:
    device = torch.device('cpu')
print(f'Using device: {device}, dtype: {dtype}')

Using device: cuda:0, dtype: torch.float32


Наша нейросеть будет обрабатывать входную последовательность по словам (word level). Мы будем использовать простую и стандартную рекуррентную архитектуру для классификации:
1. Слой представлений, превращающий id токена в вектор-эмбеддинг этого слова
2. Слой LSTM
3. Полносвязный слой, предсказывающий выход по последнему скрытому состоянию

Ниже дан код для сборки и обучения нашей нейросети.

Допишите класс-обёртку над LSTM для задачи классификации. 
**Не используйте циклы.**

**Для каждого тензора в функции `forward` подпишите в комментарии его размеры**

In [36]:
from torch import nn

In [37]:
emb_dump = None
inp_dump = None

In [38]:
class RNNClassifier(torch.nn.Module):
    def __init__(
        self, embedding_dim, hidden_dim, output_size, vocab,
        rec_layer=torch.nn.LSTM, dropout=None, **kwargs
    ):
        super().__init__()

        self.dropout = dropout
        
        self.vocab = vocab
        self.hidden_dim = hidden_dim
        self.output_size = output_size
        self.embedding_dim = embedding_dim
        
        # Create a simple lookup table that stores embeddings of a fixed dictionary and size.
        #    Use torch.nn.Embedding. Do not forget specify padding_idx!
        # YOUR CODE HERE
        self.word_embeddings = torch.nn.Embedding(
            num_embeddings=len(vocab),
            embedding_dim=embedding_dim,
            padding_idx=vocab['<pad>'],
            device=device,
        )
        
        if dropout is not None:
            self.rnn = rec_layer(
                input_size=embedding_dim, 
                hidden_size=hidden_dim, 
                device=device,
                dropout=dropout,
                **kwargs
            )
        else:
            self.rnn = rec_layer(
                input_size=embedding_dim, 
                hidden_size=hidden_dim, 
                device=device,
                **kwargs
            )
        
        # Create linear layer for classification
        # YOUR CODE HERE
        self.output = nn.Sequential(
            nn.BatchNorm1d(hidden_dim),
            nn.Linear(hidden_dim, output_size),
        )
        # self.output = nn.Sequential(
        #     nn.Linear(hidden_dim, 64),
        #     nn.ReLU(),
        #     nn.BatchNorm1d(64),
        #     nn.Linear(64, output_size),
        # )
    
    def forward(self, tokens, tokens_lens):
        """
        :param torch.Tensor(dtype=torch.long) tokens: Batch of texts represented with tokens.
        :param torch.Tensor(dtype=torch.long) tokens_lens: Number of non-padding tokens for each object in batch.
        :return torch.Tensor(dtype=torch.long): Vector representation for each sequence in batch
        """
        # Evaluate embeddings
        # DEBUG: store last input in globals
        global inp_dump
        global emb_dump
        
        # DEBUG: store last input in globals
        inp_dump = tokens.detach()
        assert not torch.any(torch.isnan(tokens)), f"Tokens has NaNs"
        
        x = self.word_embeddings(tokens)
        # DEBUG: store last input in globals
        emb_dump = x.detach()
        
        assert not torch.any(torch.isnan(x)), f"Embeddings has NaNs"
        
        # Make forward pass through recurrent network
        # YOUR CODE HERE
        x, _ = self.rnn(x)
        
        # Pass output from rnn to linear layer 
        # Note: each object in batch has its own length 
        #     so we must take rnn hidden state after the last token for each text in batch        
        x = x[tokens_lens - 1, torch.arange(0, x.shape[1]), :]
        x = self.output(x)
        
        return x

[Исходный код LSTM](http://pytorch.org/docs/master/_modules/torch/nn/modules/rnn.html#LSTM)

Допишите функции для обучения и оценки модели:

**tip:**
1. В функции `evaluate` при подсчёте метрик учитывайте, что батчи могут иметь разный размер. (в частности последний батч)

In [39]:
from tqdm.notebook import tqdm
import wandb
import random

In [40]:
def set_global_seed(seed: int):
    """
    Set global seed for reproducibility.
    """
    

    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
#     torch.use_deterministic_algorithms(True) # если нужно гарантировать 1000% воспроизводимость

    # Для Dataloader
    g = torch.Generator()
    g.manual_seed(seed)
    
    return g

# Для каждого woerker в Daaloader
def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)

In [41]:
def train_epoch(dataloader, model, loss_fn, optimizer, device):
    model.train()
    for idx, data in tqdm(enumerate(dataloader), total=len(dataloader)):
        # 1. Take data from batch
        # 2. Perform forward pass
        # 3. Evaluate loss
        # 4. Make optimizer step
        
        # labels = data['labels'].to(device)
        ratings = data['ratings'].to(device)
        tokens = data['tokens'].to(device)
        tokens_lens = data['tokens_lens'].to(device)
        
        assert not torch.any(torch.isnan(tokens)), f"Eval epoch {idx} has NaNs"
        
        optimizer.zero_grad()
        logits = model(tokens, tokens_lens)
        loss = loss_fn(logits, ratings)
        loss.backward()
        optimizer.step()
        

@torch.no_grad()
def evaluate(dataloader, model, loss_fn, device):
    model.eval()
    
    total_loss = 0.0
    total_accuracy = 0.0
    
    for idx, data in enumerate(dataloader):
        # 1. Take data from batch
        # 2. Perform forward pass
        # 3. Evaluate loss
        # 4. Evaluate accuracy
        
        # labels = data['labels'].to(device)
        ratings = data['ratings'].to(device)
        tokens = data['tokens'].to(device)
        tokens_lens = data['tokens_lens'].to(device)
        
        assert not torch.any(torch.isnan(tokens)), f"Eval epoch {idx} has NaNs"
        
        logits = model(tokens, tokens_lens)
        loss = loss_fn(logits, ratings)
        total_loss += loss
        
        pred = torch.argmax(logits, dim=1)
        total_accuracy += torch.sum(pred == ratings).item()
        
    return total_loss / len(dataloader.dataset), total_accuracy / len(dataloader.dataset)
    

def train(
    train_loader, test_loader, model, loss_fn, optimizer, device, num_epochs, name,
):
    wandb.init(project="MMP_prac_rnn_deb", name=name)
    
    test_losses = []
    train_losses = []
    test_accuracies = []
    train_accuracies = []
    
    rng = tqdm(range(num_epochs))
    
    for epoch in rng:
        train_epoch(train_loader, model, loss_fn, optimizer, device)
        
        train_loss, train_acc = evaluate(train_loader, model, loss_fn, device)
        train_accuracies.append(train_acc)
        train_losses.append(train_loss)
        
        test_loss, test_acc = evaluate(test_loader, model, loss_fn, device)
        test_accuracies.append(test_acc)
        test_losses.append(test_loss)
        
        wandb.log({"eval/loss": test_loss, "eval/accuracy": test_acc}, step=epoch)
        wandb.log({"train/loss": train_loss, "train/accuracy": train_acc}, step=epoch)
        
        print(
            'Epoch: {0:d}/{1:d}. Loss (Train/Test): {2:.3f}/{3:.3f}. Accuracy (Train/Test): {4:.3f}/{5:.3f}'.format(
                epoch + 1, num_epochs, train_losses[-1], test_losses[-1], train_accuracies[-1], test_accuracies[-1]
            )
        )
    
    wandb.finish()
    
    return train_losses, train_accuracies, test_losses, test_accuracies

Создадим модель:

In [42]:
g = set_global_seed(42)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, drop_last=False, num_workers=0, collate_fn=collate_fn_)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, num_workers=0, collate_fn=collate_fn_, 
                              shuffle=True, drop_last=True, worker_init_fn=seed_worker, generator=g)

torch.cuda.empty_cache()

In [41]:
model = RNNClassifier(
    embedding_dim=embedding_dim, hidden_dim=hidden_dim, output_size=10, vocab=vocab,
    rec_layer=torch.nn.LSTM, dropout=None
).to(device)

Создадим класс для подсчёта функции потерь и оптимизатор:

In [42]:
loss_fn = torch.nn.CrossEntropyLoss(reduction='mean')
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

Попробуем обучить модель:

**Сохраните все метрики и время работы модели. Это потребуется в конце первой части для построения графиков обучения и сравнения времени работы для всех моделей в этой секции**

In [43]:
train_losses_pure, train_accuracies_pure, test_losses_pure, test_accuracies_pure = train(
    train_dataloader, test_dataloader, model, loss_fn, optimizer, device, num_epochs,
    name="Basic RNN ratings",
)

wandb: Currently logged in as: bogachevv. Use `wandb login --relogin` to force relogin


  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/390 [00:00<?, ?it/s]

Epoch: 1/15. Loss (Train/Test): 0.030/0.031. Accuracy (Train/Test): 0.272/0.262


  0%|          | 0/390 [00:00<?, ?it/s]

...

  0%|          | 0/390 [00:00<?, ?it/s]

Epoch: 14/15. Loss (Train/Test): 0.010/0.035. Accuracy (Train/Test): 0.776/0.343


  0%|          | 0/390 [00:00<?, ?it/s]

Epoch: 15/15. Loss (Train/Test): 0.009/0.037. Accuracy (Train/Test): 0.817/0.317


eval/accuracy,▁▅▅▆▇▇█▇▇▆▆▇▅▆▄
eval/loss,▄▂▂▂▁▁▁▁▂▂▃▄▅▆█
train/accuracy,▁▂▂▃▃▃▄▅▅▅▆▆▇▇█
train/loss,█▇▆▆▆▅▅▄▄▄▃▃▂▁▁
eval/accuracy,0.31728
eval/loss,0.03729
train/accuracy,0.81728
train/loss,0.00877


Нерегуляризованные LSTM часто быстро переобучаются (и мы это видим по точности на контроле). Чтобы с этим бороться, часто используют *L2-регуляризацию* и *дропаут*.
Однако способов накладывать дропаут на рекуррентный слой достаточно много, и далеко не все хорошо работают. По [ссылке](https://medium.com/@bingobee01/a-review-of-dropout-as-applied-to-rnns-72e79ecd5b7b) доступен хороший обзор дропаутов для RNN.

Мы реализуем два варианта DropOut для RNN (и третий дополнительно). Заодно увидим, что для реализации различных усовершенствований рекуррентной архитектуры приходится "вскрывать" слой до различной "глубины".

## `Реализация дропаута по статье Гала и Гарамани. Variational Dropout (1 балл)`

Начнем с дропаута, описанного в [статье Гала и Гарамани](https://arxiv.org/abs/1512.05287).
Для этого нам потребуется перейти от использования слоя `torch.nn.LSTM`, полностью скрывающего от нас рекуррентную логику, к использованию слоя `torch.nn.LSTMCell`, обрабатывающего лишь один временной шаг нашей последовательности (а всю логику вокруг придется реализовать самостоятельно). 

Допишите класс `RNNLayer`. При `dropout=0` ваш класс должен работать как обычный слой LSTM, а при `dropout > 0` накладывать бинарную маску на входной и скрытый вектор на каждом временном шаге, причем эта маска должна быть одинаковой во все моменты времени.

Дропаут Гала и Гарамани в виде формул (m обозначает маску дропаута):

$$
h_{t-1} = h_{t-1}*m_h, \, x_t = x_t * m_x
$$

Далее обычный шаг рекуррентной архитектуры, например, LSTM:

$$
i = \sigma(h_{t-1}W^i + x_t U^i+b_i) \quad
o = \sigma(h_{t-1}W^o + x_t U^o+b_o) 
$$
$$
f = \sigma(h_{t-1}W^f + x_t U^f+b_f) \quad 
g = tanh(h_{t-1} W^g + x_t U^g+b_g) 
$$
$$
c_t = f \odot c_{t-1} +  i \odot  g \quad
h_t =  o \odot tanh(c_t)
$$

In [43]:
from typing import Union, Optional, Tuple, List

In [44]:
@torch.no_grad()
def init_h0_c0(num_objects: int, hidden_size: int, some_existing_tensor: torch.Tensor):
    """
    return h0 and c0, use some_existing_tensor.new_zeros() to gen them
    h0 shape: num_objects x hidden_size
    c0 shape: num_objects x hidden_size
    """
    h0 = some_existing_tensor.new_zeros((num_objects, hidden_size))
    c0 = some_existing_tensor.new_zeros((num_objects, hidden_size))
    
    return h0, c0

In [45]:
@torch.no_grad()
def gen_dropout_mask(input_size, hidden_size, is_training, p, some_existing_tensor):
    """
    is_training: if True, gen masks from Bernoulli
                 if False, gen masks consisting of (1-p)
    
    return dropout masks of size input_size, hidden_size if p is not None
    return one masks if p is None
    """
    if p is None:
        return some_existing_tensor.new_ones((input_size, hidden_size))
    
    if is_training:
        return some_existing_tensor.new_empty((input_size, hidden_size)).bernoulli_(1 - p)
    else:
        return some_existing_tensor.new_full((input_size, hidden_size), fill_value=1 - p)

Допишите класс-обёртку над `LSTMCell` для реализации Variational Dropout. **Используйте только цикл по времени**

**Для каждого тензора в функции `forward` подпишите в комментарии его размеры**

In [46]:
class RNNLayer(torch.nn.Module):
    def __init__(self, input_size, hidden_size, dropout: Optional[float] = None, device: Optional[torch.device] = None):
        super().__init__()

        self.dropout = dropout
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.device = device if device is not None else torch.device('cpu')
        
        self.rnn_cell = torch.nn.LSTMCell(self.input_size, self.hidden_size)
        
    def forward(self, x):
        # Initialize h_0, c_0
        h, c = init_h0_c0(
            num_objects=x.shape[1],
            hidden_size=self.hidden_size,
            some_existing_tensor=x
        )
        
        h = h.to(self.device)
        c = c.to(self.device)
        
        # Gen masks for input and hidden state
        p = self.dropout
        
        input_mask = gen_dropout_mask(
            input_size=x.shape[1],
            hidden_size=self.input_size,
            is_training=self.training,
            p=p,
            some_existing_tensor=x,
        ).to(device)
        
        hidden_st_mask = gen_dropout_mask(
            input_size=x.shape[1],
            hidden_size=self.hidden_size,
            is_training=self.training,
            p=p,
            some_existing_tensor=x,
        ).to(device)
                
        # Implement recurrent logic and return what nn.LSTM returns
        # Do not forget to apply generated dropout masks!
        
        rs = x.new_empty((x.shape[0], x.shape[1], self.hidden_size)).to(self.device)
        
        for idx in range(x.shape[0]):
            # print(f"DEB: {x.shape=}\t{h.shape=}\t{hidden_st_mask.shape=}\t{input_mask.shape=}")
#             inp = x[idx] * input_mask.broadcast_to((x.shape[1], x.shape[2]))
#             h = h * hidden_st_mask.broadcast_to((x.shape[1], self.hidden_size))
            
            inp = x[idx] * input_mask
            h = h * hidden_st_mask
    
            h, c = self.rnn_cell(inp, (h, c))
            rs[idx] = h
        
        return rs, (h, c)

Протестируйте реализованную модель с выключенным дропаутом (слой `RNNLayer` надо передать в `RNNClassifier` в качестве `rec_layer`). Замерьте время обучения. Сильно ли оно увеличилось по сравнению с `torch.nn.LSTM` (LSTM "из коробки")?

**Сохраните все метрики и время работы модели. Это потребуется в конце первой части для построения графиков обучения и сравнения времени работы для всех моделей в этой секции**

In [45]:
wandb.finish()

In [46]:
g = set_global_seed(42)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, drop_last=False, num_workers=0, collate_fn=collate_fn_)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, num_workers=0, collate_fn=collate_fn_, 
                              shuffle=True, drop_last=True, worker_init_fn=seed_worker, generator=g)

torch.cuda.empty_cache()

model = RNNClassifier(
    embedding_dim=embedding_dim, hidden_dim=hidden_dim, output_size=10, vocab=vocab,
    rec_layer=RNNLayer, dropout=0.0,
).to(device)

loss_fn = torch.nn.CrossEntropyLoss(reduction='mean')
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [47]:
train_losses_pure, train_accuracies_pure, test_losses_pure, test_accuracies_pure = train(
    train_dataloader, test_dataloader, model, loss_fn, optimizer, device, num_epochs,
    name="DEB RNN ratings DO=0",
)

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/390 [00:00<?, ?it/s]


KeyboardInterrupt



Протестируйте полученную модель с `dropout=0.25`, вновь замерив время обучения. Получилось ли побороть переобучение? Сильно ли дольше обучается данная модель по сравнению с предыдущей? (доп. время тратится на генерацию масок дропаута).

In [ ]:
g = set_global_seed(42)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, drop_last=False, num_workers=0, collate_fn=collate_fn_)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, num_workers=0, collate_fn=collate_fn_, 
                              shuffle=True, drop_last=True, worker_init_fn=seed_worker, generator=g)

torch.cuda.empty_cache()

model = RNNClassifier(
    embedding_dim=embedding_dim, hidden_dim=hidden_dim, output_size=10, vocab=vocab,
    rec_layer=RNNLayer, dropout=0.25,
).to(device)

loss_fn = torch.nn.CrossEntropyLoss(reduction='mean')
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
train_losses_pure, train_accuracies_pure, test_losses_pure, test_accuracies_pure = train(
    train_dataloader, test_dataloader, model, loss_fn, optimizer, device, num_epochs,
    name="RNN ratings DO=0.25",
)

## `Реализация дропаута по статье Гала и Гарамани. Дубль 2 (1 балл)`

<начало взлома pytorch>

При разворачивании цикла по времени средствами python обучение рекуррентной нейросети сильно замедляется. Однако для реализации дропаута Гала и Гарамани необязательно явно задавать в коде умножение нейронов на маски. Можно схитрить и обойтись использованием слоя `torch.nn.LSTM`: перед вызовом `forward` слоя `torch.nn.LSTM` подменять его веса на веса, домноженные по строкам на маски. А обучаемые веса хранить отдельно. Именно так этот дропаут реализован в библиотеке `fastai`, код из которой использован в ячейке ниже.

Такой слой реализуется в виде обертки над `torch.nn.LSTM`. Допишите класс:

In [47]:
import warnings

In [48]:
class FastRNNLayer(torch.nn.Module):
    def __init__(self, input_size, hidden_size, dropout=0.0, layers_dropout=0.0, num_layers=1, device: Optional[torch.device] = None):
        super().__init__()

        self.input_size = input_size
        self.hidden_size = hidden_size

        self.num_layers = num_layers

        self.dropout = dropout
        self.layers_dropout = layers_dropout
        self.module = torch.nn.LSTM(input_size, hidden_size, dropout=layers_dropout, num_layers=num_layers, device=device)

        self.layer_names = []
        self.layer_name_tuples = []
        for layer_n in range(self.num_layers):
            self.layer_names += [f'weight_hh_l{layer_n}', f'weight_ih_l{layer_n}']
            self.layer_name_tuples.append((f'weight_hh_l{layer_n}', f'weight_ih_l{layer_n}'))
        
        for layer in self.layer_names:
            # Get torch.nn.Parameter with weights from torch.nn.LSTM instance
            w = getattr(self.module, layer)

            # Remove it from model
            delattr(self.module, layer)

            # And create new torch.nn.Parameter with the same data but different name
            self.register_parameter(f'{layer}_raw', torch.nn.Parameter(w.data))

            # Note. In torch.nn.LSTM.forward parameter with name `layer` will be used
            #     so we must initialize it using `layer_raw` before forward pass

    @torch.no_grad()
    def _setweights(self, x):
        """
            Apply dropout to the raw weights.
        """
        
        for layer in self.layer_names:
            # Generate mask
            
            # Get torch.nn.Parameter with weights
            raw_w = getattr(self, f'{layer}_raw')
            
            # Apply dropout mask
            if 'ih' in layer:
                input_mask = gen_dropout_mask(
                    input_size=raw_w.shape[0],
                    hidden_size=raw_w.shape[1],
                    is_training=self.training,
                    p=self.layers_dropout,
                    some_existing_tensor=x,
                )
                
                # print(f"DEB: {raw_w.shape=}\t{input_mask.shape=}")
                
                masked_raw_w = raw_w * input_mask
            elif 'hh' in layer:
                hidden_st_mask = gen_dropout_mask(
                    input_size=raw_w.shape[0],
                    hidden_size=raw_w.shape[1],
                    is_training=self.training,
                    p=self.dropout,
                    some_existing_tensor=x,
                )
                
                # print(f"DEB: {raw_w.shape=}\t{hidden_st_mask.shape=}")
                
                masked_raw_w = raw_w * hidden_st_mask
            else:
                assert False, "WHF???"
            
            # Set modified weights in its place
            setattr(self.module, layer, masked_raw_w)

    def forward(self, x, h_c: Optional[Tuple[torch.Tensor, torch.Tensor]]=None):
        """
        :param x: tensor containing the features of the input sequence.
        :param Optional[Tuple[torch.Tensor, torch.Tensor]] h_c: initial hidden state and initial cell state
        """
        with warnings.catch_warnings():
            # To avoid the warning that comes because the weights aren't flattened.
            warnings.simplefilter("ignore")

            # Set new weights of self.module and call its forward
            # Pass h_c with x if it is not None. Otherwise pass only x
           
            self._setweights(x)  # set weights from layer_raw to layer
            
            if h_c is not None:
                return self.module.forward(x, h_c)
            else:
                return self.module.forward(x)
            
    def reset(self):
        if hasattr(self.module, 'reset'):
            self.module.reset()

Протестируйте реализованную модель с выключенным дропаутом (слой `FastRNNLayer` надо передать в `RNNClassifier` в качестве `rec_layer`). Замерьте время обучения. Убедитесь, что модель выдаёт такое же качество, как и оригинальная реализация LSTM.

**Сохраните все метрики и время работы модели. Это потребуется в конце первой части для построения графиков обучения и сравнения времени работы для всех моделей в этой секции**

In [56]:
g = set_global_seed(42)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, drop_last=False, num_workers=0, collate_fn=collate_fn_)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, num_workers=0, collate_fn=collate_fn_, 
                              shuffle=True, drop_last=True, worker_init_fn=seed_worker, generator=g)

torch.cuda.empty_cache()
wandb.finish()

model = RNNClassifier(
    embedding_dim=embedding_dim, hidden_dim=hidden_dim, output_size=10, vocab=vocab,
    rec_layer=FastRNNLayer, dropout=0.0,
).to(device)

loss_fn = torch.nn.CrossEntropyLoss(reduction='mean')
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [57]:
train_losses_pure, train_accuracies_pure, test_losses_pure, test_accuracies_pure = train(
    train_dataloader, test_dataloader, model, loss_fn, optimizer, device, num_epochs,
    name="FastRNN ratings DO=0",
)

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/390 [00:00<?, ?it/s]

Epoch: 1/15. Loss (Train/Test): 0.031/0.032. Accuracy (Train/Test): 0.234/0.232


  0%|          | 0/390 [00:00<?, ?it/s]

Epoch: 2/15. Loss (Train/Test): 0.031/0.031. Accuracy (Train/Test): 0.256/0.247


...

  0%|          | 0/390 [00:00<?, ?it/s]

Epoch: 14/15. Loss (Train/Test): 0.020/0.026. Accuracy (Train/Test): 0.493/0.382


  0%|          | 0/390 [00:00<?, ?it/s]

Epoch: 15/15. Loss (Train/Test): 0.020/0.026. Accuracy (Train/Test): 0.499/0.380


eval/accuracy,▁▂▃▄▆▇▆▇▇█▇████
eval/loss,██▇▆▃▂▂▂▁▁▁▁▁▁▂
train/accuracy,▁▂▂▃▅▅▅▆▆▇▇▇███
train/loss,██▇▆▅▄▄▃▃▂▂▂▁▁▁
eval/accuracy,0.3804
eval/loss,0.02591
train/accuracy,0.49924
train/loss,0.01992


Протестируйте полученный слой (вновь подставив его в `RNNClassifier` в качестве `rec_layer`) с `dropout=0.25`. Сравните время обучения с предыдущими моделями. Проследите, чтобы качество получилось такое же, как при первой реализации этого дропаута.

In [58]:
g = set_global_seed(42)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, drop_last=False, num_workers=0, collate_fn=collate_fn_)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, num_workers=0, collate_fn=collate_fn_, 
                              shuffle=True, drop_last=True, worker_init_fn=seed_worker, generator=g)

torch.cuda.empty_cache()
wandb.finish()

model = RNNClassifier(
    embedding_dim=embedding_dim, hidden_dim=hidden_dim, output_size=10, vocab=vocab,
    rec_layer=FastRNNLayer, dropout=0.25,
).to(device)

loss_fn = torch.nn.CrossEntropyLoss(reduction='mean')
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [59]:
train_losses_pure, train_accuracies_pure, test_losses_pure, test_accuracies_pure = train(
    train_dataloader, test_dataloader, model, loss_fn, optimizer, device, num_epochs,
    name="FastRNN ratings DO=0.25",
)

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/390 [00:00<?, ?it/s]

Epoch: 1/15. Loss (Train/Test): 0.031/0.032. Accuracy (Train/Test): 0.233/0.229


  0%|          | 0/390 [00:00<?, ?it/s]

Epoch: 2/15. Loss (Train/Test): 0.031/0.031. Accuracy (Train/Test): 0.253/0.246


...

  0%|          | 0/390 [00:00<?, ?it/s]

Epoch: 14/15. Loss (Train/Test): 0.021/0.026. Accuracy (Train/Test): 0.473/0.373


  0%|          | 0/390 [00:00<?, ?it/s]

Epoch: 15/15. Loss (Train/Test): 0.020/0.026. Accuracy (Train/Test): 0.484/0.370


eval/accuracy,▁▂▃▄▆▇▇▇▇▇█▇███
eval/loss,██▇▆▃▂▂▁▁▁▁▃▁▁▁
train/accuracy,▁▂▂▃▅▅▅▆▆▇▇▆▇██
train/loss,██▇▆▅▄▄▃▃▂▂▃▂▁▁
eval/accuracy,0.36968
eval/loss,0.02558
train/accuracy,0.48428
train/loss,0.02048


</конец взлома pytorch>

## `Реализация дропаута по статье Семениуты и др. (1 балл)`

Перейдем к реализации дропаута для LSTM по статье [Semeniuta et al](http://www.aclweb.org/anthology/C16-1165). 

Этот метод применения дропаута не менее популярен, чем предыдущий. Его особенность состоит в том, что он придуман специально для гейтовых архитектур. В контексте LSTM этот дропаут накладывается только на информационный поток ($m_h$ — маска дропаута):
$$
i = \sigma(h_{t-1}W^i + x_t U^i+b_i) \quad
o = \sigma(h_{t-1}W^o + x_t U^o+b_o) 
$$
$$
f = \sigma(h_{t-1}W^f + x_t U^f+b_f) \quad 
g = tanh(h_{t-1} W^g + x_t U^g+b_g) 
$$
$$
c_t = f \odot c_{t-1} +  i \odot g \odot {\bf m_h} \quad
h_t =  o \odot tanh(c_t)
$$
На входы $x_t$ маска накладывается как в предыдущем дропауте. Впрочем, на входы маску можно наложить вообще до вызова рекуррентного слоя.

Согласно статье, маска дропаута может быть как одинаковая, так и разная для всех моментов времени. Мы сделаем одинаковую для всех моментов времени.

Для реализации этого дропаута можно: 
1. самостоятельно реализовать LSTM (интерфейса LSTMCell не хватит) 
2. снова воспользоваться трюком с установкой весов (но тут мы опираемся на свойство $tanh(0)=0$, к тому же, трюк в данном случае выглядит менее тривиально, чем с дропаутом Гала). 

Предлагается реализовать дропаут по сценарию 1. Допишите класс:

**Для каждого тензора в функции `forward` подпишите в комментарии его размеры**

In [67]:
class HandmadeLSTM(torch.nn.Module):
    def __init__(self, input_size, hidden_size, dropout=0.0, device: Optional[torch.device] = None):
        super().__init__()
        
        self.device = torch.device('cpu') if device is None else device
        self.dropout = dropout
        self.input_size = input_size
        self.hidden_size = hidden_size
        
        self.input_weights = torch.nn.Linear(input_size, 4 * hidden_size).to(self.device)
        self.hidden_weights = torch.nn.Linear(hidden_size, 4 * hidden_size).to(self.device)
        self.activations = nn.ModuleList([
            nn.ReLU(), # i
            nn.ReLU(), # o
            nn.ReLU(), # f
            nn.Tanh(), # g
            nn.Tanh(), # h
        ]).to(self.device)
        
        self.reset_params()

    def reset_params(self):
        """
        Initialization as in Pytorch. 
        Do not forget to call this method!
        https://pytorch.org/docs/stable/_modules/torch/nn/modules/rnn.html#LSTM
        """
        stdv = 1.0 / np.sqrt(self.hidden_size)
        for weight in self.parameters():
            torch.nn.init.uniform_(weight, -stdv, stdv)

    def forward(self, x):
        assert not torch.any(torch.isnan(x)), f"x has nan: {x=}"
        # Use functions init_h0_c0 and gen_dropout_masks defined above
        seq_len = x.shape[0]
        batch_size = x.shape[1]
        input_dim = x.shape[2]
        
        h, c = init_h0_c0(
            num_objects=batch_size,
            hidden_size=self.hidden_size,
            some_existing_tensor=x,
        )
        
        # input_mask, hidden_st_mask = gen_dropout_mask(
        #     input_size=input_dim,
        #     hidden_size=hidden_dim,
        #     is_training=self.training,
        #     p=self.dropout,
        #     some_existing_tensor=x,
        # )
        
        # print(f"DEB: {torch.all(input_mask == 1.0)}\t{torch.all(hidden_st_mask == 1.0)}")
        
        # Implement recurrent logic to mimic torch.nn.LSTM
        # Do not forget to apply dropout mask
        rs = x.new_empty((seq_len, batch_size, self.hidden_size))
        for idx in range(seq_len):
            # x ~ [L, B, F], h ~ [H]
            # print(f"DEB: {x.shape=}\t{h.shape=}\t{c.shape=}")
            
            # DEB: disable masks
            # inp = x[idx, :, :] * input_mask # x --> [B, F]
            inp = x[idx, :, :]
            assert not torch.any(torch.isnan(inp)), f"inp has nan: idx={idx}\n{inp=}"
            
            inp4 = self.input_weights(inp)  # x4 ~ [B, 4H]
            h4 = self.hidden_weights(h)  # h4 ~ [4H]
            assert not torch.any(torch.isnan(inp4)), f"inp4 has nan: idx={idx}\n{inp4=}"
            assert not torch.any(torch.isnan(h4)), f"h4 has nan: idx={idx}\n{h4=}"
            
            y = inp4 + h4.broadcast_to((batch_size, 4 * self.hidden_size))
            assert not torch.any(torch.isnan(y)), f"y has nan: idx={idx}\n{y=}"
            
            i = self.activations[0](y[:, 0:self.hidden_size])
            o = self.activations[1](y[:, 1*self.hidden_size:2*self.hidden_size])
            f = self.activations[2](y[:, 2*self.hidden_size:3*self.hidden_size])
            g = self.activations[3](y[:, 3*self.hidden_size:4*self.hidden_size])
            
            assert not torch.any(torch.isnan(i)), f"i has nan: idx={idx}\n{i=}"
            assert not torch.any(torch.isnan(o)), f"o has nan: idx={idx}\n{o=}"
            assert not torch.any(torch.isnan(f)), f"f has nan: idx={idx}\n{f=}"
            assert not torch.any(torch.isnan(g)), f"g has nan: idx={idx}\n{g=}"
            
            # DEB: disable masks
            # c = f * c + i * g * hidden_st_mask
            c = f * c + i * g
            h = o * self.activations[4](c)
            assert not torch.any(torch.isnan(c)), f"c has nan: idx={idx}\n{c=}"
            assert not torch.any(torch.isnan(h)), f"h has nan: idx={idx}\n{h=}"
            
            rs[idx, :, :] = h
        
        assert not torch.any(torch.isnan(rs)), f"rs has nan: {rs=}"
        return rs, (h, c)

Протестируйте вашу реализацию без дропаута (проконтролируйте качество и сравните время обучения с временем обучения `torch.nn.LSTM` и `RNNLayer`), а также с `dropout=0.25`. Сравните качество модели с таким дропаутом с качеством модели с дропаутом Гала и Гарамани.

**Сохраните все метрики и время работы модели. Это потребуется в конце первой части для построения графиков обучения и сравнения времени работы для всех моделей в этой секции**

In [68]:
wandb.finish()

In [69]:
g = set_global_seed(42)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, drop_last=False, num_workers=0, collate_fn=collate_fn_)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, num_workers=0, collate_fn=collate_fn_, 
                              shuffle=True, drop_last=True, worker_init_fn=seed_worker, generator=g)

torch.cuda.empty_cache()
wandb.finish()

model = RNNClassifier(
    embedding_dim=embedding_dim, hidden_dim=hidden_dim, output_size=10, vocab=vocab,
    rec_layer=HandmadeLSTM, dropout=0.0,
).to(device)

loss_fn = torch.nn.CrossEntropyLoss(reduction='mean')
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [70]:
train_losses_pure, train_accuracies_pure, test_losses_pure, test_accuracies_pure = train(
    train_dataloader, test_dataloader, model, loss_fn, optimizer, device, num_epochs,
    name="HandmadeLSTM ratings DO=0.0",
)

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/390 [00:00<?, ?it/s]

Epoch: 1/15. Loss (Train/Test): 0.030/0.032. Accuracy (Train/Test): 0.264/0.233


  0%|          | 0/390 [00:00<?, ?it/s]

Epoch: 2/15. Loss (Train/Test): 0.029/0.031. Accuracy (Train/Test): 0.293/0.242


  0%|          | 0/390 [00:00<?, ?it/s]

AssertionError: Embeddings has NaNs

In [73]:
inp_dump = inp_dump.cpu()

torch.unique(inp_dump)

tensor([   0,    1,    4,  ..., 5001, 5004, 5016])

In [76]:
model = model.cpu()

In [82]:
for i in range(len(vocab)):
    assert not torch.any(
        torch.isnan(
            model.word_embeddings(
                torch.LongTensor([i])
            )       
        )
    ), f"Token: {i}"

AssertionError: Token: 1

In [83]:
model.word_embeddings.weight

Parameter containing:
tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [    nan,     nan,     nan,  ...,     nan,     nan,     nan],
        [ 1.1044, -1.1028,  0.5543,  ..., -0.6463,  0.0749,  0.1939],
        ...,
        [ 1.0837, -0.6411,  1.0597,  ...,  1.3895, -0.3897,  0.6304],
        [ 0.4510, -0.0567, -1.1788,  ..., -0.0786,  2.0267,  0.6129],
        [-0.6576, -0.7521,  0.2827,  ..., -0.0446,  1.5368, -0.1816]],
       requires_grad=True)

In [ ]:
# YOUR CODE HERE
...

## `Сравнение всех предложенных моделей (1 балл)`

Используя замеры времени заполните табличку с временем работы четырёх реализованных моделей в следующей ячейке:

| torch.nn.LSTM | RNNLayer | FastRNNLayer | HandmadeLSTM |
|---------------|----------|--------------|--------------|
| 2m 35s        | 14m 16s  | 2m 41s       | 31m 44s      |

In [ ]:
import matplotlib.pyplot as plt

Крайне желательно рисовать графики в векторном формате. 

Если по каким-то причинам, отрисовка не будет работать, закомментируйте следующую ячейку.

In [ ]:
%matplotlib inline

import matplotlib_inline
from IPython.display import set_matplotlib_formats

matplotlib_inline.backend_inline.set_matplotlib_formats('pdf', 'svg')

Нарисуйте два графика — функция потерь и качество на обучающей и тестовой выборке для всех 7 моделей обученных выше.

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(10, 10))

# YOUR CODE HERE
...

axes[0].legend()
axes[0].grid(True)
axes[0].set_xlabel('Epoch')
axes[0].set_title('CrossEntropy Loss')

axes[1].legend()
axes[1].grid(True)
axes[1].set_xlabel('Epoch')
axes[1].set_title('Accuracy')

fig.tight_layout()
plt.show()

Сделайте итоговые выводы о качестве работы моделей с разными реализациями DropOut:

**Ответ:**

## `Бонус. Zoneout (0.5 балла)`

Это еще одна модификация идеи дропаута применительно к рекуррентным нейросетям. В Zoneout на каждом временном шаге с вероятностью $p$ компонента скрытого состояния обновляется, а с вероятностью $1-p$ берется с предыдущего шага. 
В Виде формул ($m^t_h$ - бинарная маска):
 
(сначала обычный рекуррентный переход, например LSTM)
$$
i = \sigma(h_{t-1}W^i + x_t U^i+b_i) \quad
o = \sigma(h_{t-1}W^o + x_t U^o+b_o) 
$$
$$
f = \sigma(h_{t-1}W^f + x_t U^f+b_f) \quad 
g = tanh(h_{t-1} W^g + x_t U^g+b_g) 
$$
$$
c_t = f \odot c_{t-1} +  i \odot  g \quad
h_t =  o \odot tanh(c_t)
$$
Затем Zoneout:
$$
h_t = h_t * m_h^t + h_{t-1}*(1-m_h^t)
$$
В этом методе маска уже должна быть разная во все моменты времени (иначе метод упрощается до дропаута Гала и Гарамани). На входы $x_t$ вновь можно накладывать маску до начала работы рекуррентного слоя.  

Если у вас осталось время, вы можете реализовать этот метод. Выберите основу из трех рассмотренных случаев самостоятельно.

**Полный балл ставится только при наличии качественного и количественного сравнения с предыдущими моделями.**

# `Часть 2. Language Modeling с помощью LSTM (5 баллов)`

Во второй части мы попробуем обучить модель для генерации отзывов по их началу.

Концептуально модель будет выглядеть следующим образом:
    
![image info](https://blog.feedly.com/wp-content/uploads/2019/03/Screen-Shot-2019-03-06-at-12.08.35-PM.png)

В процессе обучения будем тренировать сеть предсказывать вероятность следующего символа при условии всех предыдущих. Эту вероятность можно моделировать с помощью скрытого состояния $h^{(t)}$ пропуская его через линейный слой с выходной размерностью равной размерности словаря:
$$
p(x^{t}|x^{t-1}, ..., x^{1}) = SoftMax(Linear(h^{(t)}))
$$

Обратите внимание, что для вычисления $p(x^{t}|x^{t-1}, ..., x^{1})$ для всех моментов времени достаточно сделать один проход по RNN, а затем применить линейное преобразование ко всем скрытым состояниям.

В качестве функции потерь необходимо использовать `CrossEntropy`.

Рассмотрим другой важный момент. Для того, чтобы решить данную задачу, модель должна уметь определять момент начала генерации предложения и оповещать о завершении генерации — конце предложения. Для этого добавим в словарь вспомогательные токены `<sos>`, `<eos>`. Добавив `<sos>` в начало каждого предложения и `<eos>` в конец.

Модель сможет начинать генерацию как только ей будет передан токен `<sos>` и заканчивать генерацию, как только на очередном месте самым вероятным токеном оказывается `<eos>`.

Для решения этой задачи мы воспользуемся уже реализованной LSTM с дропаутом `FastRNNLayer` и классом `RNNClassifier`, то есть архитектура сети принципиально не поменяется. 

## `Реализация модели и цикла обучения (2 балла)`

**Не используйте циклы в `RNNLM`, `LMCrossEntropyLoss`, `LMAccuracy`**

In [182]:
class RNNLM(RNNClassifier):
    def __init__(
        self, embedding_dim, hidden_dim, vocab, dropout=0.5, layers_dropout=0.5, num_layers=1
    ):
        super().__init__(
            embedding_dim=embedding_dim, hidden_dim=hidden_dim, output_size=len(vocab), vocab=vocab,
            rec_layer=FastRNNLayer, dropout=dropout, layers_dropout=layers_dropout, num_layers=num_layers
        )
    
        # super().__init__(
        #     embedding_dim=embedding_dim, hidden_dim=hidden_dim, output_size=len(vocab), vocab=vocab,
        #     rec_layer=nn.LSTM, dropout=None, num_layers=num_layers
        # )
        
        self.linear = nn.Sequential(
            nn.Linear(hidden_dim, len(vocab))
        )
    
    def forward(self, tokens, tokens_lens):
        """
        :param torch.Tensor(dtype=torch.long) tokens: 
            Batch of texts represented with tokens. Shape: [T, B]
        :param torch.Tensor(dtype=torch.long) tokens_lens: 
            Number of non-padding tokens for each object in batch. Shape: [B]
        :return torch.Tensor: 
            Distribution of next token for each time step. Shape: [T, B, V], V — size of vocabulary
        """
        # Make embeddings for all tokens
        x = self.word_embeddings(tokens)
        
        # Forward pass embeddings through network
        x, _ = self.rnn(x)
        
        # Take all hidden states from the last layer of LSTM for each step and perform linear transformation
        x = self.linear(x)
        
        return x

Реализуем функцию потерь для данной задачи. 

Моменты на которые нужно обратить внимание:
1. Распределение вероятности следующего токена для последнего токена в последовательности не участвует в подсчёте функции потерь.
2. Необходимо учитывать, что в одном батче могут быть тексты разной длины.

Для решения второй проблемы можно воспользоваться функцией `torch.nn.utils.rnn.pack_padded_sequence`. 

Принимая на вход батч тензоров и длину каждого тензора без учёта паддинга эта функция позволяет получить все элементы в тензорах, которые не относятся к паддингу в виде плоского массива:

In [183]:
padded_tensors = torch.tensor([
    [[1, 11, 111], [2, 22, 222], [3, 33, 333]],
    [[4, 44, 444], [5, 55, 555], [6, 66, 666]],
    [[7, 77, 777], [0, 0, 0], [8, 88, 888]],
    [[9, 99, 999], [0, 0, 0], [0, 0, 0]]
])
tensors_lens = torch.tensor([4, 2, 3])

Обратите внимание, что `torch.nn.utils.rnn.pack_padded_sequence` автоматически переупорядочивает тензоры в батче по убыванию их длины.

In [184]:
torch.nn.utils.rnn.pack_padded_sequence(padded_tensors, tensors_lens, batch_first=False, enforce_sorted=False)[0].shape

torch.Size([9, 3])

In [185]:
class LMCrossEntropyLoss(torch.nn.CrossEntropyLoss):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        
    def forward(self, outputs, tokens, tokens_lens):
        """
        :param torch.Tensor outputs: Output from RNNLM.forward. Shape: [T, B, V]
        :param torch.Tensor tokens: Batch of tokens. Shape: [T, B]
        :param torch.Tensor tokens_lens: Length of each sequence in batch
        :return torch.Tensor: CrossEntropyLoss between corresponding logits and tokens
        """
        # Use torch.nn.utils.rnn.pack_padded_sequence().data to remove padding and flatten logits and tokens
        # Do not forget specify enforce_sorted=False and correct value of batch_first 
        
        # print(f"DEB: {outputs.shape=}\t{tokens.shape=}")
        
        # packed_outputs = torch.nn.utils.rnn.pack_padded_sequence(outputs[:-1], tokens_lens.cpu()-1, batch_first=False, enforce_sorted=False)[0]
        # packed_tokens = torch.nn.utils.rnn.pack_padded_sequence(tokens[1:], tokens_lens.cpu()-1, batch_first=False, enforce_sorted=False)[0]
        
        packed_outputs = torch.nn.utils.rnn.pack_padded_sequence(outputs, tokens_lens, batch_first=False, enforce_sorted=False)[0]
        packed_tokens = torch.nn.utils.rnn.pack_padded_sequence(tokens, tokens_lens, batch_first=False, enforce_sorted=False)[0]
        
        # print(f"DEB: {packed_outputs.shape=}\t{packed_tokens.shape=}")
        
        # Use super().forward(..., ...) to compute CrossEntropyLoss
        return super().forward(
            input=packed_outputs,
            target=packed_tokens
        )

Для оценки качества нам также необходимо вычислять долю правильно предсказанных токенов. Реализуйте класс для вычисления точности.

In [186]:
class LMAccuracy(torch.nn.Module):
    def __init__(self):
        super().__init__()
        
    def forward(self, outputs, tokens, tokens_lens):
        """
        :param torch.Tensor outputs: Output from RNNLM.forward. Shape: [T, B, V]
        :param torch.Tensor tokens: Batch of tokens. Shape: [T, B]
        :param torch.Tensor tokens_lens: Length of each sequence in batch
        :return torch.Tensor: Accuracy for given logits and tokens
        """
        # Use torch.nn.utils.rnn.pack_padded_sequence().data to remove padding and flatten logits and tokens
        # Do not forget specify enforce_sorted=False and correct value of batch_first 
        # YOUR CODE HERE
        
        # packed_outputs = torch.nn.utils.rnn.pack_padded_sequence(outputs, tokens_lens.cpu(), batch_first=False, enforce_sorted=False)[0]
        # packed_tokens = torch.nn.utils.rnn.pack_padded_sequence(tokens, tokens_lens.cpu(), batch_first=False, enforce_sorted=False)[0]
    
        packed_outputs = torch.nn.utils.rnn.pack_padded_sequence(outputs, tokens_lens, batch_first=False, enforce_sorted=False)[0]
        packed_tokens = torch.nn.utils.rnn.pack_padded_sequence(tokens, tokens_lens, batch_first=False, enforce_sorted=False)[0]
    
        preds = torch.argmax(packed_outputs, dim=1)
    
        return torch.sum(
            preds == packed_tokens
        )

Модифицируйте функции `train_epoch`, `evaluate`, `train` для обучения LM.

**При вычислении точности, обратите внимание на то, что мы не предсказываем первый токен в каждой последовательности и токены, относящиеся к паддингу.**

In [187]:
def train_epoch_lm(dataloader, model, loss_fn, optimizer, device):
    model.train()
    for idx, data in tqdm(enumerate(dataloader), total=len(dataloader)):
        # 1. Take data from batch
        # 2. Perform forward pass
        # 3. Evaluate loss
        # 4. Make optimizer step
        
        optimizer.zero_grad()
        
        tokens = data['tokens'][:-1, :].to(device)
        targets = data['tokens'][1:, :].to(device)
        tokens_lens = data['tokens_lens'].cpu()
        
        logits = model(tokens, tokens_lens - 1)
        loss = loss_fn(logits, targets, tokens_lens - 1)
        loss.backward()
        
        optimizer.step()

@torch.no_grad()
def evaluate_lm(dataloader, model, loss_fn, device):
    model.eval()
    
    total_tokens = 0
    total_loss = 0.0
    total_accuracy = 0.0
    
    accuracy_fn = LMAccuracy()  # DEB: return 0.0
    
    for idx, data in enumerate(dataloader):
        # 1. Take data from batch
        # 2. Perform forward pass
        # 3. Evaluate loss
        # 4. Evaluate accuracy
        
        tokens = data['tokens'][:-1, :].to(device)
        targets = data['tokens'][1:, :].to(device)
        tokens_lens = data['tokens_lens'].cpu()
        
        logits = model(tokens, tokens_lens - 1)
        loss = loss_fn(logits, targets, tokens_lens - 1).item()
        acc = accuracy_fn(logits, targets, tokens_lens - 1).item()
        
        total_loss += loss
        total_tokens += torch.sum(tokens_lens)
        total_accuracy += acc
        
            
    return total_loss / total_tokens, total_accuracy / total_tokens

def train_lm(
    train_loader, test_loader, model, loss_fn, optimizer, device, num_epochs
):
    test_losses = []
    train_losses = []
    test_accuracies = []
    train_accuracies = []
    for epoch in tqdm(range(num_epochs)):
        train_epoch_lm(train_loader, model, loss_fn, optimizer, device)
        
        train_loss, train_acc = evaluate_lm(train_loader, model, loss_fn, device)
        train_accuracies.append(train_acc)
        train_losses.append(train_loss)
        
        test_loss, test_acc = evaluate_lm(test_loader, model, loss_fn, device)
        test_accuracies.append(test_acc)
        test_losses.append(test_loss)
        
        print(
            'Epoch: {0:d}/{1:d}. Loss (Train/Test): {2:.3f}/{3:.3f}. Accuracy (Train/Test): {4:.3f}/{5:.3f}'.format(
                epoch + 1, num_epochs, train_losses[-1], test_losses[-1], train_accuracies[-1], test_accuracies[-1]
            )
        )
    return train_losses, train_accuracies, test_losses, test_accuracies

Теперь у нас всё готово для обучения модели.

Создадим словарь с `<sos>`, `<eos>` токенами.

Обратите внимание, что в отличие от классификации текстов нам необходимо значительно увеличить размер словаря, чтобы доля `<unk>` токенов была не велика.

Так же, так как задача генерации значительно сложнее задачи классификации текстов будем обучать модель только на префиксах рецензий длины $20$. Это позволяет значительно ускорить обучение.

In [70]:
specials = ['<pad>', '<unk>', '<sos>', '<eos>']
for special in specials:
    counter[special] = 0
# min_freq=8 is approximately equivalent to max_size=30000. 
#   You can lower min_freq in order to make model vocabulary more diverse 
lm_vocab = torchtext.vocab.vocab(counter, specials=specials, special_first=True, min_freq=8)
lm_vocab.set_default_index(vocab['<unk>'])

In [71]:
lm_datasets_dump_path = pathlib.Path("./lm_datasets_dump.bin")

if not lm_datasets_dump_path.exists():
    lm_test_dataset = LargeMovieReviewDataset(test_data_path, lm_vocab, max_len=20, pad_sos=True, pad_eos=True)
    lm_train_dataset = LargeMovieReviewDataset(train_data_path, lm_vocab, max_len=20, pad_sos=True, pad_eos=True)
    
    with open(lm_datasets_dump_path, "wb") as f:
        torch.save(
            obj=(lm_test_dataset, lm_train_dataset),
            f=f
        )
else:
    with open(lm_datasets_dump_path, "rb") as f:
        lm_test_dataset, lm_train_dataset = torch.load(f)

In [72]:
lm_test_dataset[0]

{'text': "Once again Mr. Costner has dragged out a movie for far longer than necessary. Aside from the terrific sea rescue sequences, of which there are very few I just did not care about any of the characters. Most of us have ghosts in the closet, and Costner's character are realized early on, and then forgotten until much later, by which time I did not care. The character we should really care about is a very cocky, overconfident Ashton Kutcher. The problem is he comes off as kid who thinks he's better than anyone else around him and shows no signs of a cluttered closet. His only obstacle appears to be winning over Costner. Finally when we are well past the half way point of this stinker, Costner tells us all about Kutcher's ghosts. We are told why Kutcher is driven to be the best with no prior inkling or foreshadowing. No magic here, it was all I could do to keep from turning it off an hour in.",
 'label': tensor(0),
 'rating': tensor(1),
 'tokens': tensor([ 2,  4,  5,  6,  7,  8,  

Создадим даталоадеры для тестовой и обучающей выборок:

In [174]:
lm_test_dataloader = DataLoader(
    lm_test_dataset, batch_size=196, shuffle=False, num_workers=0, 
    collate_fn=partial(collate_fn, padding_value=lm_vocab.lookup_indices(['<pad>'])[0])
)
lm_train_dataloader = DataLoader(
    lm_train_dataset, batch_size=196, shuffle=True, num_workers=0, 
    collate_fn=partial(collate_fn, padding_value=lm_vocab.lookup_indices(['<pad>'])[0])
)

Убедитесь, что все предложения имеют в начале `<sos>` токен, а в конце — `<eos>` токен.

In [175]:
batch = next(iter(lm_train_dataloader))
batch['tokens'], batch['tokens_lens']

(tensor([[    2,     2,     2,  ...,     2,     2,     2],
         [ 3236,  1768,     7,  ...,    50, 10243,   431],
         [ 1385,   369,  2059,  ...,    78,    35,  1096],
         ...,
         [ 7065,     1,  3278,  ...,  3694,   325,  1886],
         [ 4045,  3826,    94,  ...,   495,    24,  1223],
         [    3,     3,     3,  ...,     3,     3,     3]]),
 tensor([22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22,
         22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22,
         22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22,
         22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22,
         22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22,
         22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22,
         22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22,
         22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22

Создадим модель, функцию потерь и оптимизатор: 

In [176]:
import gc

In [177]:
torch.cuda.empty_cache()
gc.collect()

2486

In [178]:
lm_model = RNNLM(
    embedding_dim=512, hidden_dim=512, vocab=lm_vocab, dropout=0.6, layers_dropout=0.6, num_layers=2
).to(device=device)

# lm_model = RNNLM(
#     embedding_dim=512, hidden_dim=512, vocab=lm_vocab, num_layers=2
# ).to(device=device)

In [179]:
lm_loss_fn = LMCrossEntropyLoss(reduction='sum')
lm_optimizer = torch.optim.Adam(lm_model.parameters(), lr=0.005, weight_decay=1.2e-6)

Обучим модель:

In [180]:
# lm_model = torch.compile(lm_model)

In [181]:
lm_train_losses, lm_train_accuracies, lm_test_losses, lm_test_accuracies = train_lm(
    lm_train_dataloader, lm_test_dataloader, lm_model, lm_loss_fn, lm_optimizer, device, 10
)

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/128 [00:00<?, ?it/s]

DEB: x.shape=torch.Size([21, 196, 512])
DEB: x.shape=torch.Size([21, 196, 512])
DEB: x.shape=torch.Size([21, 196, 512])
DEB: x.shape=torch.Size([21, 196, 512])
DEB: x.shape=torch.Size([21, 196, 512])
DEB: x.shape=torch.Size([21, 196, 512])
DEB: x.shape=torch.Size([21, 196, 512])
DEB: x.shape=torch.Size([21, 196, 512])
DEB: x.shape=torch.Size([21, 196, 512])
DEB: x.shape=torch.Size([21, 196, 512])
DEB: x.shape=torch.Size([21, 196, 512])
DEB: x.shape=torch.Size([21, 196, 512])
DEB: x.shape=torch.Size([21, 196, 512])
DEB: x.shape=torch.Size([21, 196, 512])
DEB: x.shape=torch.Size([21, 196, 512])
DEB: x.shape=torch.Size([21, 196, 512])
DEB: x.shape=torch.Size([21, 196, 512])
DEB: x.shape=torch.Size([21, 196, 512])
DEB: x.shape=torch.Size([21, 196, 512])
DEB: x.shape=torch.Size([21, 196, 512])
DEB: x.shape=torch.Size([21, 196, 512])
DEB: x.shape=torch.Size([21, 196, 512])
DEB: x.shape=torch.Size([21, 196, 512])
DEB: x.shape=torch.Size([21, 196, 512])
DEB: x.shape=torch.Size([21, 196, 512])


KeyboardInterrupt: 

In [96]:
lm_model_dump_path = pathlib.Path("./lm_model_dump.bin")

with open(lm_model_dump_path, "wb") as f:
    torch.save(
        obj=lm_model,
        f=f
    )

In [190]:
with open(lm_model_dump_path, "rb") as f:
    lm_model = torch.load(f)

In [192]:
pred_0 = lm_model(batch['tokens'].to(device), batch['tokens_lens'].to(device))[:, 1, :].cpu()
pred_0 = torch.argmax(pred_0, dim=1)

In [193]:
pred_0, batch['tokens'][:, 1]

(tensor([  7,   7,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1, 169,   1,
           1,   1,   3,   3,   3,   3,   3,   3]),
 tensor([    2,  1768,   369,  1205,  7630,   502,  1363, 12547,   524,  2417,
          5326,    76,  1109,  9018,     1,  1168,   657,   657,     1,     1,
          3826,     3]))

## `Реализация декодера (1 балл)`

Теперь, реализуем последнюю деталь — декодирование с использованием обученной модели.
Есть несколько вариантов. Рассмотрим два самых простых:
1. **Жадное декодирование.** На каждом шаге мы выбираем токен с максимальной вероятностью и используем его для обновления скрытого состояния RNN.
2. **Top-k sampling.** На очередном шаге рассматриваются $k$ токенов с самыми большими вероятностями. Остальные токены игнорируются. Из выбранных токенов семплируется следующий токен пропорционально их вероятностям.

Прочитать подробнее про разные варианты декодирования можно по ссылкам:
1. [От huggingface](https://huggingface.co/blog/how-to-generate)
2. [На towardsdatascience](https://towardsdatascience.com/decoding-strategies-that-you-need-to-know-for-response-generation-ba95ee0faadc)

Существенным в процессе декодирования является критерий останова. Как только очередной самый вероятный символ оказался `<eos>`, то данная последовательность считается сгенерированной. Однако, может так оказаться, что `<eos>` никогда не будет выбран, тогда необходимо прекратить генерацию, как только длина последовательности перейдёт порог `max_generated_len`.

In [102]:
@torch.no_grad()
def decode(model, start_tokens, start_tokens_lens, max_generated_len=20, top_k=None):
    """
    :param RNNLM model: Model
    :param torch.Tensor start_tokens: Batch of seed tokens. Shape: [T, B]
    :param torch.Tensor start_tokens_lens: Length of each sequence in batch. Shape: [B]
    :param int max_generated_len: Maximum lenght of generated samples
    :param Optional[int] top_k: Number of tokens with the largest probability to sample from
    :return Tuple[torch.Tensor, torch.Tensor]. 
        Newly predicted tokens and length of generated part. Shape [T*, B], [B]
    """
    # Get embedding for start_tokens
    embedding = model.word_embeddings(start_tokens)
    
    # Pass embedding through rnn and collect hidden states and cell states for each time moment
    all_h, all_c = [], []
    h = embedding.new_zeros([model.rnn.num_layers, start_tokens.shape[1], model.hidden_dim])
    c = embedding.new_zeros([model.rnn.num_layers, start_tokens.shape[1], model.hidden_dim])
    for time_step in range(start_tokens.shape[0]):
        _, (h, c) = model.rnn(embedding[time_step][None, :, :], (h, c))
        all_h.append(h)
        all_c.append(c)
    
    all_h = torch.stack(all_h, dim=1)
    all_c = torch.stack(all_c, dim=1)
    
    # Take final hidden state and cell state for each start sequence in batch
    # We will use them as h_0, c_0 for generation new tokens
    h = all_h[:, start_tokens_lens - 1, torch.arange(start_tokens_lens.shape[0])]
    c = all_c[:, start_tokens_lens - 1, torch.arange(start_tokens_lens.shape[0])]
    
    # List of predicted tokens for each time step
    predicted_tokens = []
    # Length of generated part for each object in the batch
    decoded_lens = torch.zeros_like(start_tokens_lens, dtype=torch.long)
    # Boolean mask where we store if the sequence has already generated
    # i.e. `<eos>` was selected on any step
    is_finished_decoding = torch.zeros_like(start_tokens_lens, dtype=torch.bool)
    
    # Stop when all sequences in the batch are finished
    while not torch.all(is_finished_decoding) and torch.max(decoded_lens) < max_generated_len:
        # Evaluate next token distribution using hidden state h.
        # Note. Over first dimension h has hidden states for each layer of LSTM.
        #     We must use hidden state from the last layer
        # logits, (h, c) = model.rnn(h, (h, c))
        # logits = model.linear(logits)
        
        logits = model.linear(h[-1, :, :])
        
        if top_k is not None:
            # Top-k sampling. Use only top-k most probable logits to sample next token
            indices_to_remove = logits < torch.topk(logits, top_k)[0][..., -1, None]
            # Mask non top-k logits
            logits[indices_to_remove] = -1e10
            # Sample next_token. 
            logits = F.softmax(logits, dim=-1)
            next_token = torch.multinomial(
                input=logits,
                num_samples=1,
            )[:, 0]
        else:
            # Select most probable token
            next_token = torch.argmax(logits, dim=-1)
        
        predicted_tokens.append(next_token)
        
        decoded_lens += (~is_finished_decoding)
        is_finished_decoding |= (next_token == torch.tensor(model.vocab.lookup_indices(['<eos>'])[0]))

        # Compute embedding for next token
        embedding = model.word_embeddings(next_token)
        
        # Update hidden and cell states
        _, (h, c) = model.rnn(embedding[None, :, :], (h, c))
    
    return torch.stack(predicted_tokens), decoded_lens

Попробуем сгенерировать продолжения для нескольких префиксов:

In [103]:
start_tokens = torch.tensor([
    lm_model.vocab.lookup_indices(['<sos>', '<pad>', '<pad>', '<pad>']),
    lm_model.vocab.lookup_indices(['<sos>', 'my', 'favorite', 'movie']),
    lm_model.vocab.lookup_indices(['<sos>', 'the', 'best', 'movie']),
    lm_model.vocab.lookup_indices(['<sos>', 'the', 'worst', 'movie']),
]).T

start_tokens_lens = torch.tensor([1, 4, 4, 4])

In [110]:
lm_model = lm_model.cpu()
lm_model.eval()
# decoded_tokens, decoded_lens = decode(lm_model, start_tokens, start_tokens_lens, max_generated_len=10, top_k=20)
# decoded_tokens, decoded_lens = decode(lm_model, start_tokens, start_tokens_lens, max_generated_len=10, top_k=5)
decoded_tokens, decoded_lens = decode(lm_model, start_tokens, start_tokens_lens, max_generated_len=10, top_k=None)

In [111]:
decoded_tokens.shape, decoded_lens.shape

(torch.Size([10, 4]), torch.Size([4]))

In [112]:
for text_idx in range(start_tokens.shape[1]):
    decoded_text_tokens = decoded_tokens[:decoded_lens[text_idx], text_idx]
    tokens = start_tokens[:start_tokens_lens[text_idx], text_idx].tolist()
    tokens = tokens + decoded_text_tokens.tolist()
    print(f"{tokens=}")
    words = np.array(lm_model.vocab.get_itos())[np.array(tokens)]
    print(' '.join(words))
    
    # text = ' '.join(words).replace('<', '&lt;').replace('>', '&gt;')
    # print(f"{text}")
    # display(Markdown(f'<div class="alert alert-block alert-info"> <b>{text}</b></div>'))

tokens=[2, 7, 1, 1, 1, 1, 1, 1, 1, 1, 1]
<sos> movie <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk>
tokens=[2, 1, 1406, 7, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
<sos> <unk> favorite movie <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk>
tokens=[2, 1, 60, 7, 525, 131, 7, 136, 1, 1, 1, 1, 1, 1]
<sos> <unk> best movie ever seen movie made <unk> <unk> <unk> <unk> <unk> <unk>
tokens=[2, 1, 1203, 7, 525, 131, 7, 136, 1, 1, 1, 1, 1, 1]
<sos> <unk> worst movie ever seen movie made <unk> <unk> <unk> <unk> <unk> <unk>


Попробуйте выполнить семплирование для разных $k$. Сравните результаты top-k семплирования с жадным декодированием. Опишите ваши наблюдения.

In [113]:
decoded_tokens, decoded_lens = decode(lm_model, start_tokens, start_tokens_lens, max_generated_len=10, top_k=15)
for text_idx in range(start_tokens.shape[1]):
    decoded_text_tokens = decoded_tokens[:decoded_lens[text_idx], text_idx]
    tokens = start_tokens[:start_tokens_lens[text_idx], text_idx].tolist()
    tokens = tokens + decoded_text_tokens.tolist()
    print(f"{tokens=}")
    words = np.array(lm_model.vocab.get_itos())[np.array(tokens)]
    print(' '.join(words))
    
    # text = ' '.join(words).replace('<', '&lt;').replace('>', '&gt;')
    # print(f"{text}")
    # display(Markdown(f'<div class="alert alert-block alert-info"> <b>{text}</b></div>'))

tokens=[2, 7, 525, 1891, 131, 7, 136, 169, 93, 241, 28]
<sos> movie ever heard seen movie made like one movies time
tokens=[2, 1, 1406, 7, 525, 131, 7, 169, 169, 371, 93, 60, 3017, 93]
<sos> <unk> favorite movie ever seen movie like like love one best flicks one
tokens=[2, 1, 60, 7, 1, 136, 1, 115, 131, 159, 159, 174, 28, 28]
<sos> <unk> best movie <unk> made <unk> film seen first first minutes time time
tokens=[2, 1, 1203, 7, 525, 130, 131, 131, 88, 7, 1, 7, 689, 689]
<sos> <unk> worst movie ever ive seen seen times movie <unk> movie bad bad


In [114]:
decoded_tokens, decoded_lens = decode(lm_model, start_tokens, start_tokens_lens, max_generated_len=10, top_k=25)
for text_idx in range(start_tokens.shape[1]):
    decoded_text_tokens = decoded_tokens[:decoded_lens[text_idx], text_idx]
    tokens = start_tokens[:start_tokens_lens[text_idx], text_idx].tolist()
    tokens = tokens + decoded_text_tokens.tolist()
    print(f"{tokens=}")
    words = np.array(lm_model.vocab.get_itos())[np.array(tokens)]
    print(' '.join(words))
    
    # text = ' '.join(words).replace('<', '&lt;').replace('>', '&gt;')
    # print(f"{text}")
    # display(Markdown(f'<div class="alert alert-block alert-info"> <b>{text}</b></div>'))

tokens=[2, 93, 241, 525, 169, 7, 131, 88, 704, 115, 131]
<sos> one movies ever like movie seen times say film seen
tokens=[2, 1, 1406, 7, 1023, 435, 72, 1023, 159, 7, 65, 211, 1, 438]
<sos> <unk> favorite movie great director films great first movie could get <unk> make
tokens=[2, 1, 60, 7, 159, 28, 99, 1, 1, 7, 136, 3]
<sos> <unk> best movie first time watch <unk> <unk> movie made <eos>
tokens=[2, 1, 1203, 7, 131, 1, 1, 463, 1, 93, 72, 60, 525, 525]
<sos> <unk> worst movie seen <unk> <unk> horror <unk> one films best ever ever


In [115]:
decoded_tokens, decoded_lens = decode(lm_model, start_tokens, start_tokens_lens, max_generated_len=10, top_k=50)
for text_idx in range(start_tokens.shape[1]):
    decoded_text_tokens = decoded_tokens[:decoded_lens[text_idx], text_idx]
    tokens = start_tokens[:start_tokens_lens[text_idx], text_idx].tolist()
    tokens = tokens + decoded_text_tokens.tolist()
    print(f"{tokens=}")
    words = np.array(lm_model.vocab.get_itos())[np.array(tokens)]
    print(' '.join(words))
    
    # text = ' '.join(words).replace('<', '&lt;').replace('>', '&gt;')
    # print(f"{text}")
    # display(Markdown(f'<div class="alert alert-block alert-info"> <b>{text}</b></div>'))

tokens=[2, 115, 239, 159, 261, 7, 3340, 1, 261, 7, 119]
<sos> film thought first saw movie theater <unk> saw movie still
tokens=[2, 1, 1406, 7, 241, 882, 169, 7, 230, 895, 709, 169, 241, 704]
<sos> <unk> favorite movie movies terrible like movie even worse think like movies say
tokens=[2, 1, 60, 7, 525, 525, 131, 129, 65, 689, 159, 91, 99, 7]
<sos> <unk> best movie ever ever seen every could bad first dont watch movie
tokens=[2, 1, 1203, 7, 131, 882, 1, 7, 169, 7, 689, 204, 689, 147]
<sos> <unk> worst movie seen terrible <unk> movie like movie bad pretty bad script


In [116]:
decoded_tokens, decoded_lens = decode(lm_model, start_tokens, start_tokens_lens, max_generated_len=10, top_k=None)
for text_idx in range(start_tokens.shape[1]):
    decoded_text_tokens = decoded_tokens[:decoded_lens[text_idx], text_idx]
    tokens = start_tokens[:start_tokens_lens[text_idx], text_idx].tolist()
    tokens = tokens + decoded_text_tokens.tolist()
    print(f"{tokens=}")
    words = np.array(lm_model.vocab.get_itos())[np.array(tokens)]
    print(' '.join(words))
    
    # text = ' '.join(words).replace('<', '&lt;').replace('>', '&gt;')
    # print(f"{text}")
    # display(Markdown(f'<div class="alert alert-block alert-info"> <b>{text}</b></div>'))

tokens=[2, 7, 1, 1, 1, 1, 1, 1, 1, 1, 1]
<sos> movie <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk>
tokens=[2, 1, 1406, 7, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
<sos> <unk> favorite movie <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk>
tokens=[2, 1, 60, 7, 525, 131, 7, 136, 1, 1, 1, 1, 1, 1]
<sos> <unk> best movie ever seen movie made <unk> <unk> <unk> <unk> <unk> <unk>
tokens=[2, 1, 1203, 7, 525, 131, 7, 136, 1, 1, 1, 1, 1, 1]
<sos> <unk> worst movie ever seen movie made <unk> <unk> <unk> <unk> <unk> <unk>


**Ответ:**

## `Beam Search (2 балла)`

Рассмотрим более продвинутый алгоритм для декодирования. Реализуйте алгоритм Beam Search.

Несколько замечаний по имплементации:

1. При больших размерах `beam_size` число гипотез ($B \times \text{beam\_size}$) на очередном шаге может быть слишком большим. Поэтому может потребоваться разбить все гипотезы на отдельные батчи и делать forward-pass в несколько итераций. Используйте [`torch.split`](https://pytorch.org/docs/stable/generated/torch.split.html)
2. Для выбора лучших гипотез используйте [`torch.topk`](https://pytorch.org/docs/stable/generated/torch.topk.html). Обратите внимание на индексы, которые возвращает эта функция (может пригодиться метод [`torch.remainder`](https://pytorch.org/docs/stable/generated/torch.remainder.html))
3. Можно отслеживать, какие элементы в батче (или какие гипотезы) закончили генерацию. Делая forward-pass только для незавершённых гипотез, можно ускорить декодинг, однако, это усложнит реализацию

In [213]:
def print_tensor(t):
    print(f"{t.shape=}")
    print("tensor=(")
    for i in range(t.shape[0]):
        print(f"\t{t[i]}")
    print(f")")

In [348]:
@torch.no_grad()
def decode_beam_search(model, start_tokens, start_tokens_lens, max_generated_len=20, beam_size=5):
    """
    :param RNNLM model: Model
    :param torch.Tensor start_tokens: Batch of seed tokens. Shape: [T, B]
    :param torch.Tensor start_tokens_lens: Length of each sequence in batch. Shape: [B]
    :param int max_generated_len: Maximum length of generated samples
    :param int beam_size: Size of beam
    :return Tuple[torch.Tensor, torch.Tensor, torch.Tensor]. 
        Newly predicted tokens, lengths of generated parts and log probabilities for each hypotheses 
        Shape [T*, B, beam_size], [T*, beam_size], [T*, beam_size]
    """
    
    # L — number of RNN layers in the model, H — hidden size, BS — beam size
    #
    # 1. Make forward pass of start_tokens through the model. 
    #      Obtain the last cell and hidden state for each element in the batch 
    #          (i.e. tensors of shape [L, B, H])
    #      Use those states as the initialization for each hypotheses in the beam 
    #          (i.e. tensors of shape [L, B * BS, H])
    #      Initialize probabilities for each hypotheses in the beam with 1.0
    #          (i.e. tensor of shape [B * BS])
    #      Initialize vector that show whether hypothesis is finished
    #          (i.e. tensor of shape [B * BS])
    # 2. While all sequences do not end with <eos> and their length less than max_generated_len
    #      1. Get probabilities for the next token for each hypothesis 
    #          (i.e. tensor of shape [B * BS, V])
    #      2. Use those probabilities to compute probability for each extension of each hypothesis
    #          (i.e. tensor of shape [B * BS, V])
    #      3. For each element in the batch select new BS best hypotheses
    #          Note, that some of the hypotheses on the previous step have been finished
    #            so their probability should not change. So you have to select BS best hypotheses
    #            among all extension of unfinished hypotheses and finished hypotheses
    #          As a result you will have a new token for best extensions of unfinished hypotheses
    #          For simplisity you can use <EOS> token if you select finished hypothesis in the beam
    #            i.e. tensor of shape [B * BS] of indices for selected hypotheses and
    #                 tensor of shape [B * BS] of extension tokens for each hypothesis
    #      4. Update probabilities for each hypotheses and is_finished state for each hypothesis
    #          Concat new tokens to the existing prefixes
    #      5. Update hidden and cell state to correspond to the selected hypothesis

    batch_size = start_tokens.shape[1]

    eos_idx = model.vocab.get_stoi()['<eos>']
    
    # Get embeddings for the start tokens
    embedding = model.word_embeddings(start_tokens)
    
    # Make forward pass through the RNN and 
    #   obtain the last cell and hidden state for each element in the batch
    all_h, all_c = [], []
    h = embedding.new_zeros([model.rnn.num_layers, start_tokens.shape[1], model.hidden_dim])
    c = embedding.new_zeros([model.rnn.num_layers, start_tokens.shape[1], model.hidden_dim])
    for time_step in range(start_tokens.shape[0]):
        _, (h, c) = model.rnn(embedding[time_step][None, :, :], (h, c))
        all_h.append(h)
        all_c.append(c)
    
    all_h = torch.stack(all_h, dim=1)
    all_c = torch.stack(all_c, dim=1)

    start_h = all_h[:, start_tokens_lens - 1, torch.arange(start_tokens_lens.shape[0])] # [L, B, H]
    start_c = all_c[:, start_tokens_lens - 1, torch.arange(start_tokens_lens.shape[0])] # [L, B, H]

    # Use those states as the initialization for each hypotheses in the beam
    h = torch.cat([start_h] * beam_size, dim=1) # [L, B * BS, H]
    c = torch.cat([start_c] * beam_size, dim=1) # [L, B * BS, H]
    
    # Select initial tokens for each hypotheses in the beam
    #   Compute log probabilities and select top-beam_size tokens for each element
    #   Use them to initialize beam search state
    
    
    start_log_probas = F.log_softmax(model.linear(start_h[-1, :, :]), dim=-1) # [B, V]
    log_probas, new_tokens = torch.topk(
        input=start_log_probas,
        k=beam_size,
    ) # ([B, BS], [B, BS])
    log_probas = log_probas.view((-1, )) # [B * BS]
    new_tokens = new_tokens.view((-1, )) # [B * BS]
    print(f"DEB: {log_probas.shape=}\t{new_tokens.shape=}")
    # hypotesis = new_tokens.reshape((1, -1)) # [1, B * BS]
    hypotesis = torch.clone(new_tokens.reshape((1, -1))) # [1, B * BS]
    print(f"DEB: ", end="")
    print_tensor(hypotesis)
    
    is_finished = h.new_zeros((batch_size * beam_size, ), dtype=torch.bool) # [B * BS]
    decoded_lens = h.new_zeros((batch_size * beam_size, ), dtype=torch.int64) # [B * BS]

    while not torch.all(is_finished) and hypotesis.shape[0] < max_generated_len:
        embedding = model.word_embeddings(new_tokens)
        # print(f"DEB: {embedding.shape=}")
        _, (new_h, new_c) = model.rnn(embedding[None, :, :], (h, c))
        # print(f"DEB: {h.shape=}\t{new_h.shape=}")
        
        # Get probabilities for the next token for each hypothesis
        next_tokens_logits = model.linear(h[-1, :, :])
        next_tokens_logits[is_finished] = -1e-10
        next_tokens_logits[is_finished, eos_idx] = 0
        
        next_token_log_probas = F.log_softmax(next_tokens_logits, dim=-1)  # [B * BS, V]
        
        # Use those probabilities to compute probability for each extension of each hypothesis
        print(f"DEB: {next_token_log_probas.shape=}\t{log_probas.shape=}")
        # extension_log_probas = log_probas[:, None] + next_token_log_probas * (~is_finished)[:, None]  # [B * BS, V]
        extension_log_probas = log_probas[:, None] + next_token_log_probas  # [B * BS, V]

        # For each element in the batch select new BS best hypotheses
        #   You can use loop over different beams
        for batch_idx in range(batch_size):
            indices_to_process = torch.topk(
                input=extension_log_probas[batch_idx * beam_size:(batch_idx + 1) * beam_size].view((-1, )), 
                k=beam_size
            ).indices
            beam_idx = indices_to_process // extension_log_probas.shape[1]
            token_idx = indices_to_process % extension_log_probas.shape[1]
            
            print(f"Beam idx: {beam_idx}\tToken idx: {token_idx}\tRaw indices: {indices_to_process=}")
            
            new_tokens[batch_idx * beam_size:(batch_idx + 1) * beam_size] = token_idx
            
            idx = batch_idx * beam_size + beam_idx
            # hypotesis = hypotesis.swapaxes(0, 1)
            # hypotesis[batch_idx * beam_size:(batch_idx + 1) * beam_size] = hypotesis[batch_idx * beam_size:(batch_idx + 1) * beam_size][beam_idx]
            # hypotesis = hypotesis.swapaxes(0, 1)
            hypotesis[:, batch_idx * beam_size:(batch_idx + 1) * beam_size] = hypotesis[:, batch_idx * beam_size:(batch_idx + 1) * beam_size][:, beam_idx]
            
            # Update probabilities for each hypotheses and is_finished state and decoded_lens for each hypothesis
            log_probas[idx] = extension_log_probas[idx, token_idx]
            is_finished[idx] = (token_idx == eos_idx)
            
            # Update hidden and cell state to correspond to the selected hypothesis
            h[:, idx, ] = new_h[:, idx, :]
            c[:, idx, ] = new_c[:, idx, :]
            
        decoded_lens += (~is_finished)

        # Concat new tokens to the existing prefixes
        hypotesis = torch.cat([hypotesis, new_tokens.reshape((1, -1))], dim=0)
        # hypotesis = torch.cat([torch.clone(hypotesis), torch.clone(new_tokens.reshape((1, -1)))], dim=0)
        print(f"DEB: ", end="")
        print_tensor(hypotesis)     
        
    return (
        hypotesis.view(-1, start_tokens.shape[1], beam_size), 
        decoded_lens.view(start_tokens.shape[1], beam_size),
        log_probas.view(start_tokens.shape[1], beam_size)
    )

In [349]:
start_tokens = torch.tensor([
    lm_model.vocab.lookup_indices(['<sos>', '<pad>', '<pad>', '<pad>']),
    lm_model.vocab.lookup_indices(['<sos>', 'my', 'favorite', 'movie']),
    lm_model.vocab.lookup_indices(['<sos>', 'the', 'best', 'movie']),
    lm_model.vocab.lookup_indices(['<sos>', 'the', 'worst', 'movie']),
]).T

start_tokens_lens = torch.tensor([1, 4, 4, 4])

In [350]:
# lm_model.to(device).eval()
# start_tokens = start_tokens.to(device)
# start_tokens_lens = start_tokens_lens.to(device)

In [351]:
lm_model.cpu().eval()
start_tokens = start_tokens.cpu()
start_tokens_lens = start_tokens_lens.cpu()

In [352]:
beam_size = 100
# beam_size = 10
decoded_tokens, decoded_lens, log_probas = decode_beam_search(
    lm_model, start_tokens, start_tokens_lens, max_generated_len=10, beam_size=beam_size
)

DEB: log_probas.shape=torch.Size([400])	new_tokens.shape=torch.Size([400])
DEB: t.shape=torch.Size([1, 400])
tensor=(
	tensor([   7,  115,  159,  261,   93,    1,   29,  118, 2059,   91,   50,  169,
        1165,  371,  704,  579,  426, 1023,  130,   78, 1893,  872,  709, 2964,
        1123,  462, 3694,  369,  239,   95,  510,  262,  689,   87,  236,  131,
         518,  455, 1213, 1497,  874, 1203,  305,   60,   40,   99, 1385,  706,
         241, 1913, 1302,  283,   26,  190,  885,  235,  877, 1804,   28,  447,
        4740,  206,  286,  230,  212,  124,  148, 2416, 1231,  719,  502,  322,
         298, 4303,  158,   75,  141,   65,  523, 1604,  198, 1891, 1602, 3414,
         204,  788,  205, 1169, 2066,  769, 1214,    8, 2438, 1799,  964, 1963,
        1474, 1338,  368,  864,    1,  525,    7,   93,  136,  130,  115,  159,
          29,   28, 1023,  689,  426,  131,  169, 1722,  241,  455,   60,  230,
         369,   65,  471,  261,  212,   50, 2059,   91,  205,  502,  510, 1203,
 

In [353]:
decoded_lens

tensor([[9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9,
         9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9,
         9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9,
         9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9,
         9, 9, 9, 9],
        [9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9,
         9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9,
         9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9,
         9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9,
         9, 9, 9, 9],
        [9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9,
         9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9,
         9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9,
         9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9,

In [354]:
for start_tokens_elem, start_tokens_lens_elem, decoded_tokens_elem, decoded_lens_elem, log_probas_elem in zip(
    start_tokens.T, start_tokens_lens,
    decoded_tokens.permute(1, 2, 0), decoded_lens.permute(0, 1), log_probas.permute(0, 1)
):
    start_tokens_elem = start_tokens_elem[:start_tokens_lens_elem].tolist()
    start_words = np.array(lm_model.vocab.get_itos())[np.array(start_tokens_elem)]
    
    print(' '.join(start_words))
    start_text = ' '.join(start_words).replace('<', '&lt;').replace('>', '&gt;')
    # display(Markdown(f'<div class="alert alert-block alert-info"> <b>{start_text}</b></div>'))
    
    for idx, (hyp, hyp_len, hyp_log_prob) in enumerate(zip(decoded_tokens_elem, decoded_lens_elem, log_probas_elem)):
        if idx >= 3:
            break
            
        hyp = hyp[:hyp_len].tolist()
        # print(f"Tokens: {tokens}")
        hyp_words = np.array(lm_model.vocab.get_itos())[np.array(hyp)]
        print(f"Text: {' '.join(hyp_words)}, log proba: {hyp_log_prob:.4f}")
        # hyp_text = ' '.join(hyp_words).replace('<', '&lt;').replace('>', '&gt;')
        # display(Markdown(
        #     f'<div class="alert alert-block alert-success"> <b>{hyp_log_prob:.3f}: {hyp_text}</b></div>'
        # ))

<sos>
Text: liked ever movie movie <unk> seen movie made saw, log proba: -18.0324
Text: seeing ever ever movie <unk> movie bad <unk> ever, log proba: -15.7798
Text: got ever ever ever seen ever ever <unk> one, log proba: -16.8093
<sos> <unk> favorite movie
Text: life ever ive movie made seen movie <unk> seen, log proba: -18.2851
Text: based ever movie film seen seen <unk> <unk> <unk>, log proba: -18.6186
Text: classic ever <unk> seen <unk> like ever <unk> <unk>, log proba: -18.0889
<sos> <unk> best movie
Text: ever spoilers film seen ever <unk> bad seen movie, log proba: -17.8790
Text: ever good movie seen saw made movie <unk> best, log proba: -18.3225
Text: ever spoilers film seen ever <unk> bad seen movie, log proba: -18.2945
<sos> <unk> worst movie
Text: bad ever made ever seen worst seen <unk> seen, log proba: -18.1796
Text: ever spoilers ever movie <unk> <unk> made seen movie, log proba: -17.1323
Text: ever never movie seen made seen <unk> seen <unk>, log proba: -18.2516


Попробуйте выполнить декодинг для разных `beam_size`. Убедитесь, что при `beam_search=1` семплирование совпадает с top-1 (greedy decoding) подходом. 

Сравните результаты Beam Search с top-k семплированием и жадным декодированием. Опишите ваши наблюдения.

In [258]:
# YOUR CODE HERE

## `Бонус. Существенное улучшение качества (до 6 баллов)`

Та модель, которая использовалась в предыдущей части во многом заимствует улучшения LSTM из статьи [Regularizing and Optimizing LSTM Language Models](https://arxiv.org/pdf/1708.02182.pdf). Вы можете попробовать применить другие варианты регуляризации из данной статьи для существенного улучшения качества LM.

Например:
1. Dropout для эмбеддингов **(+0.25)**
2. Dropout входов и выходов RNN **(+0.25)**
3. Регуляризация активаций (AR/TAR) **(+1.0)**
4. NT-ASGD **(+1.5)**
5. Tied веса эмбеддингов и софтмакса **(+1.0)**
6. Attention **(+2.0)**

**Полные баллы ставятся только при наличии качественного и количественного сравнения с бейзлайном.**

**Для эксперимента с Attention необходимо изобразить Attention Maps для нескольких примеров.**